### This version adds the part for user initial  set up 

In [2]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [82]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
#from scipy import integrate
import statistics as stats
#import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
import sys
from geopy.geocoders import Nominatim
from geopy import distance
from geopy import Point
import statistics 

## Dataset files 

In [4]:
#reviewJson = "..\\data\\Export_CleanedReview.json"
#reviewJsonWithClosedRes = "..\\data\\Export_CleanedReviewWithClosedRes.json"
#reviewJsonToronto = "..\\data\\Export_TorontoData.json"
reviewJsonToronto = "..\\data\\Cleaned_Toronto_Reviews.json"


## Load Data
Select top frenquent user and top frequenty restaurants that had at least 1 review >= 4 stars (Kickking out users that gave all  reviews <=3 and restaurants that never got start >= 4 stars)

In [5]:
def get_yelp_df(path = 'data/', filename = 'Export_CleanedReview.json', sampling=False, top_user_num=7000, top_item_num=5000):
    """
    Get the pandas dataframe
    Sampling only the top users/items by density 
    Implicit representation applies
    """
    with open(filename,'r') as f:
        data = f.readlines()
        data = list(map(json.loads, data))
    
    data = data[0]
    #Get all the data from the dggeata file
    df = pd.DataFrame(data)
    
    df.rename(columns={'stars': 'review_stars', 'text': 'review_text', 'cool': 'review_cool',
                       'funny': 'review_funny', 'useful': 'review_useful'},
              inplace=True)

    df['business_num_id'] = df.business_id.astype('category').\
        cat.rename_categories(range(0, df.business_id.nunique()))
    df['business_num_id'] = df['business_num_id'].astype('int')

    df['user_num_id'] = df.user_id.astype('category').\
    cat.rename_categories(range(0, df.user_id.nunique()))
    df['user_num_id'] = df['user_num_id'].astype('int')

    df['timestamp'] = df['date'].apply(date_to_timestamp)

    if sampling:
        df = filter_yelp_df(df, top_user_num=top_user_num, top_item_num=top_item_num)
        # Refresh num id
        df['business_num_id'] = df.business_id.astype('category').\
        cat.rename_categories(range(0, df.business_id.nunique()))
        df['business_num_id'] = df['business_num_id'].astype('int')
        
        df['user_num_id'] = df.user_id.astype('category').\
        cat.rename_categories(range(0, df.user_id.nunique()))
        df['user_num_id'] = df['user_num_id'].astype('int')
#     drop_list = ['date','review_id','review_funny','review_cool','review_useful']
#     df = df.drop(drop_list, axis=1)

    df = df.reset_index(drop = True)

    return df 

def filter_yelp_df(df, top_user_num=7000, top_item_num=5000):
    #Getting the reviews where starts are above 3
    df_implicit = df[df['review_stars']>3]
    frequent_user_id = df_implicit['user_num_id'].value_counts().head(top_user_num).index.values
    frequent_item_id = df_implicit['business_num_id'].value_counts().head(top_item_num).index.values
    return df.loc[(df['user_num_id'].isin(frequent_user_id)) & (df['business_num_id'].isin(frequent_item_id))]

def date_to_timestamp(date):
    dt = datetime.datetime.strptime(date, '%Y-%m-%d')
    return time.mktime(dt.timetuple())

def df_to_sparse(df, row_name='userId', col_name='movieId', value_name='rating',
                 shape=None):
    rows = df[row_name]
    cols = df[col_name]
    if value_name is not None:
        values = df[value_name]
    else:
        values = [1]*len(rows)

    return csr_matrix((values, (rows, cols)), shape=shape)

## Get rating-UI and timestamp-UI matrix from original df

In [6]:
def get_rating_timestamp_matrix(df, sampling=False, top_user_num=7000, top_item_num=5000):
    """
    """
    #make the df implicit with top frenquent users and 
    #no need to sample anymore if df was sampled before 
    if sampling:
        df = filter_yelp_df(df, top_user_num=top_user_num, top_item_num=top_item_num)

    rating_matrix = df_to_sparse(df, row_name='user_num_id',
                                 col_name='business_num_id',
                                 value_name='review_stars',
                                 shape=None)
    
    #Have same dimension and data entries with rating_matrix, except that the review stars are - user avg
#     ratingWuserAvg_matrix = df_to_sparse(df, row_name='user_num_id',
#                                  col_name='business_num_id',
#                                  value_name='reviewStars_userAvg',
#                                  shape=None)
    
    timestamp_matrix = df_to_sparse(df, row_name='user_num_id',
                                    col_name='business_num_id',
                                    value_name='timestamp',
                                    shape=None)
    
    
    IC_matrix, IC_dictionary = get_I_C(df)
#     ratingWuserAvg_matrix
    return rating_matrix, timestamp_matrix, IC_matrix, IC_dictionary

In [7]:
def get_I_C(df):
    lst = df.categories.values.tolist()
    cat = []
    for i in range(len(lst)):
        if lst[i] is None:
            print(i)
        cat.extend(lst[i].split(', '))
        
    unique_cat = set(cat)
    #     set categories id
    df_cat = pd.DataFrame(list(unique_cat),columns=["Categories"])
    df_cat['cat_id'] = df_cat.Categories.astype('category').cat.rename_categories(range(0, df_cat.Categories.nunique()))
    dict_cat = df_cat.set_index('Categories')['cat_id'].to_dict()
    
    df_I_C = pd.DataFrame(columns=['business_num_id', 'cat_id'])
    
    for i in range((df['business_num_id'].unique().shape)[0]):
        df_temp = df[df['business_num_id'] == i].iloc[:1]
        temp_lst = df_temp['categories'].to_list()[0].split(",")
        for j in range(len(temp_lst)):
            df_I_C = df_I_C.append({'business_num_id' : i  , 'cat_id' : dict_cat[temp_lst[j].strip()]} , ignore_index=True)
    
    IC_Matrix = df_to_sparse(df_I_C, row_name='business_num_id',
                                 col_name='cat_id',
                                 value_name=None,
                                 shape=None)    
    return IC_Matrix, dict_cat

# Construct IC, IP, IR, ID... for Critiquing

In [8]:
def get_IP_matrix_dictionary(df,item_sim):
    # get an initial item price dataframe(without onehot encoding)
    # drop duplicates
    df_temp = df[['business_num_id', 'price']].drop_duplicates()
    # with nontype with string "NaN"
    df_temp.fillna(value = "NaN", inplace = True)
    
    for i in tqdm(range(df_temp.shape[0])):
    # find all the items with no price
        if df_temp[df_temp['business_num_id'] == i]['price'].values[0] == "NaN":
            # get the index of the second large number in the similarity matrix
            temp_l = list(item_sim[i])
            index = [temp_l.index(x) for x in sorted(temp_l, reverse=True)[:2]][1]
            # get the dollar sign of the similar item
            dollar_of_sim_item = df_temp[df_temp['business_num_id'] == index]['price'].values[0]
            # replace the Nan
            df_temp.loc[df_temp['business_num_id'] == i,"price"] = dollar_of_sim_item

    # assign single dollar sign($) to the ones still with no price tag(since there is no items that are similar to this item)
    df_temp.loc[df_temp["price"] == "NaN","price"] = "$"

    # One hot encoding
    #note that the last column is price__$$$$
    #cat_columns = ["price"]
    #df_processed = pd.get_dummies(df_temp, prefix_sep="_",
    #                          columns=cat_columns)
    
    df_processed = df_temp.copy()
    df_processed['Price'] = df_processed.apply (lambda row: len(row.price), axis=1)
    
    #drop the $ column
    df_processed = df_processed.drop('price', 1)
    
    #Adding additional column of price label, range 1-4
    #df_preprocessed['Price_label'] = df.apply (lambda row: label_price(row), axis=1)
    df_processed.set_index("business_num_id", drop=True, inplace=True)
    I_P_dictionary = df_processed.to_dict()['Price']
    df_processed.reset_index(level=0, inplace=True)
    
    return df_processed, I_P_dictionary

In [9]:
def get_IS_dictionary(df):
    df_IS = df[['business_num_id', 'business_stars']].drop_duplicates()
    df_IS.set_index("business_num_id", drop=True, inplace=True)
    IS_dictionary = df_IS.to_dict()['business_stars']
    df_IS.reset_index(level=0, inplace=True)
    
    return IS_dictionary

In [10]:
# Input a list of prediction matrix
def get_ID_dictionary(df,prediction_matrix,intersection):
    ID_dictionary = dict()
    length = len(prediction_matrix)
    
    for j in tqdm(range(length)):
        #Save the coordinates of the business id to a dictionary 
        coordinateDict = yaml.safe_load(df[df["business_num_id"] == prediction_matrix[j]].iloc[0].coordinates)
    
        #Load the business latitude and longitude
        test_point = Point(coordinateDict['latitude'],coordinateDict['longitude'])

        #Get the distance with the test point, unit in km 
        result = round(distance.distance(intersection,test_point).kilometers,1)

        ID_dictionary[prediction_matrix[j]] = result
        
    return ID_dictionary

## Time ordered split 

In [11]:
def time_ordered_splitModified(rating_matrix, ratingWuserAvg_matrix, timestamp_matrix, ratio=[0.5, 0.2, 0.3],
                       implicit=True, remove_empty=False, threshold=3,
                       sampling=False, sampling_ratio=0.1, trainSampling=1):
    """
    Split the data to train,valid,test by time
    ratio:  train:valid:test
    threshold: for implicit representation
    """
    
    
    if implicit:
        temp_rating_matrix = sparse.csr_matrix(rating_matrix.shape)
        temp_rating_matrix[(rating_matrix > threshold).nonzero()] = 1
        rating_matrix = temp_rating_matrix
        timestamp_matrix = timestamp_matrix.multiply(rating_matrix)
        #ratingWuserAvg_matrix = ratingWuserAvg_matrix.multiply(rating_matrix)

    nonzero_index = None

    #Default false, not removing empty columns and rows
    #Should not have this case, since users should have at least 1 record of 4,5 
    #And restuarant should have at least 1 record of 4,5 
    if remove_empty:
        # Remove empty columns. record original item index
        nonzero_index = np.unique(rating_matrix.nonzero()[1])
        rating_matrix = rating_matrix[:, nonzero_index]
        timestamp_matrix = timestamp_matrix[:, nonzero_index]
        ratingWuserAvg_matrix = ratingWuserAvg_matrix[:, nonzero_index]

        # Remove empty rows. record original user index
        nonzero_rows = np.unique(rating_matrix.nonzero()[0])
        rating_matrix = rating_matrix[nonzero_rows]
        timestamp_matrix = timestamp_matrix[nonzero_rows]
        ratingWuserAvg_matrix = ratingWuserAvg_matrix[nonzero_rows]

    user_num, item_num = rating_matrix.shape

    rtrain = []
    rtrain_userAvg = []
    rtime = []
    rvalid = []
    rvalid_userAvg = []
    rtest = []
    rtest_userAvg = []
    # Get the index list corresponding to item for train,valid,test
    item_idx_train = []
    item_idx_valid = []
    item_idx_test = []
    
    for i in tqdm(range(user_num)):
        #Get the non_zero indexs, restuarants where the user visited/liked if implicit 
        item_indexes = rating_matrix[i].nonzero()[1]        
        #Get the data for the user
        data = rating_matrix[i].data      
        #Get time stamp value 
        timestamp = timestamp_matrix[i].data 
        #Get review stars with user avg data 
        if implicit == False:
            dataWuserAvg = ratingWuserAvg_matrix[i].data

            
        #Non zero reviews for this user
        num_nonzeros = len(item_indexes)
        
        #If the user has at least one review
        if num_nonzeros >= 1:
            num_test = int(num_nonzeros * ratio[2])
            num_valid = int(num_nonzeros * (ratio[1] + ratio[2]))
            valid_offset = num_nonzeros - num_valid
            
            # Adding this for sampling for training set
            valid_offsetSample = int(valid_offset*trainSampling)
            test_offset = num_nonzeros - num_test
            
            #Sort the timestamp for each review for the user
            argsort = np.argsort(timestamp)
            
            #Sort the reviews for the user according to the time stamp 
            data = data[argsort]
            
            #Sort the review with user avg accoridng to the time stamp
            if implicit == False:
                dataWuserAvg = dataWuserAvg[argsort]
            
            #Non-zero review index sort according to time
            item_indexes = item_indexes[argsort]
            
            #list of ratings, num of valid_offset index, index where there's non-zeros
            #if take from old to new
            #rtrain.append([data[:valid_offsetSample], np.full(valid_offsetSample, i), item_indexes[:valid_offsetSample]])
            #if take from new to old
            rtrain.append([data[valid_offset-valid_offsetSample:valid_offset], np.full(valid_offsetSample, i), item_indexes[valid_offset-valid_offsetSample:valid_offset]])
            rvalid.append([data[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                           item_indexes[valid_offset:test_offset]])
            rtest.append([data[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
            
            if implicit == False:
                #Now for the rating matrix that considers user average rating
                #list of ratings, num of valid_offset index, index where there's non-zeros
                #from old to new
                #rtrain_userAvg.append([dataWuserAvg[:valid_offsetSample], np.full(valid_offsetSample, i), item_indexes[:valid_offsetSample]])
                #take nearest
                rtrain_userAvg.append([dataWuserAvg[valid_offset-valid_offsetSample:valid_offset], np.full(valid_offsetSample, i), item_indexes[valid_offset-valid_offsetSample:valid_offset]])                
                    
                rvalid_userAvg.append([dataWuserAvg[valid_offset:test_offset], np.full(test_offset - valid_offset, i),
                               item_indexes[valid_offset:test_offset]])
                
                rtest_userAvg.append([dataWuserAvg[test_offset:], np.full(num_test, i), item_indexes[test_offset:]])
                
            item_idx_train.append(item_indexes[:valid_offsetSample])
            item_idx_valid.append(item_indexes[:test_offset])
            item_idx_test.append(item_indexes[test_offset:])
            
        else:
            item_idx_train.append([])
    
    rtrain = np.array(rtrain)
    rvalid = np.array(rvalid)
    rtest = np.array(rtest)
   
    if implicit == False:
        rtrain_userAvg = np.array(rtrain_userAvg)
        rvalid_userAvg = np.array(rvalid_userAvg)
        rtest_userAvg = np.array(rtest_userAvg)

    #take non-zeros values, row index, and column (non-zero) index and store into sparse matrix
    rtrain = sparse.csr_matrix((np.hstack(rtrain[:, 0]), (np.hstack(rtrain[:, 1]), np.hstack(rtrain[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rvalid = sparse.csr_matrix((np.hstack(rvalid[:, 0]), (np.hstack(rvalid[:, 1]), np.hstack(rvalid[:, 2]))),
                               shape=rating_matrix.shape, dtype=np.float32)
    rtest = sparse.csr_matrix((np.hstack(rtest[:, 0]), (np.hstack(rtest[:, 1]), np.hstack(rtest[:, 2]))),
                              shape=rating_matrix.shape, dtype=np.float32)
    
    if implicit == False:
        rtrain_userAvg = sparse.csr_matrix((np.hstack(rtrain_userAvg[:, 0]), (np.hstack(rtrain_userAvg[:, 1]), np.hstack(rtrain_userAvg[:, 2]))),
                                   shape=rating_matrix.shape, dtype=np.float32)
        rvalid_userAvg = sparse.csr_matrix((np.hstack(rvalid_userAvg[:, 0]), (np.hstack(rvalid_userAvg[:, 1]), np.hstack(rvalid_userAvg[:, 2]))),
                                   shape=rating_matrix.shape, dtype=np.float32)
        rtest_userAvg = sparse.csr_matrix((np.hstack(rtest_userAvg[:, 0]), (np.hstack(rtest_userAvg[:, 1]), np.hstack(rtest_userAvg[:, 2]))),
                                  shape=rating_matrix.shape, dtype=np.float32)

    return rtrain, rvalid, rtest,rtrain_userAvg, rvalid_userAvg, rtest_userAvg, nonzero_index, timestamp_matrix, item_idx_train, item_idx_valid, item_idx_test


### Popularity Calculation

In [12]:
def get_three_popularity_matrix(df_original,rtrain):
    # get the list of popular items by ranking the number of reviews
    numUsers = rtrain.shape[0]
    numItems = rtrain.shape[1]
    
    dff_popular = df_original.copy()
    dff_popular = dff_popular.sort_values(by=["review_count_y"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
    popular_list_num_of_reviews = dff_popular["business_num_id"].tolist()
    
    # get the list of popular items by ranking average rating score
    dff_popular_rating = df_original.copy()
    dff_popular_rating = dff_popular_rating.sort_values(by=["business_stars"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
    popular_list_avg_stars = dff_popular_rating["business_num_id"].tolist()
    
    lst_temp = []
    for item in tqdm(range(numItems)):
        numOfUsersRated = len(rtrain.toarray()[:, item].nonzero()[0])
        if numOfUsersRated <= 50:
            lst_temp.append(item)
    popular_list_avg_stars = [x for x in popular_list_avg_stars if x not in lst_temp]
    
    # get the popularity items by using the percentage liked method(number of liked items / total items)
    predictionMatrix = np.zeros((numUsers , numItems))

    # Define function for converting 1-5 rating to 0/1 (like / don't like)
    vf = np.vectorize(lambda x: 1 if x >= 4 else 0)
    rtrain_array = rtrain.toarray()
    # For every item calculate the number of people liked (4-5) divided by the number of people that rated
    itemPopularity = np.zeros((numItems))
    for item in range(numItems):
        numOfUsersRated = len(rtrain_array[:, item].nonzero()[0])
        numOfUsersLiked = len(vf(rtrain_array[:, item]).nonzero()[0])
#         if numOfUsersRated == 0:
        # set a threshold to filter out restaurants with very few reviews
        if numOfUsersRated <= 30:
            itemPopularity[item] = 0
        else:
            itemPopularity[item] = numOfUsersLiked/numOfUsersRated
    popular_list_liked_ratio = itemPopularity.argsort()
    
    return np.asarray(popular_list_num_of_reviews),np.asarray(popular_list_avg_stars),popular_list_liked_ratio

## Get df for training corpus

In [13]:
#Item idex matrix stores the reivews starts
#This function returns a list of index for the reviews included in training set 
def get_corpus_idx_list(df, item_idx_matrix):
    """
    Input: 
    df: total dataframe
    item_idx_matrix: train index list got from time_split 
    Output: row index in original dataframe for training data by time split
    """
    lst = []
    #For all the users: 5791
    for i in tqdm(range(len(item_idx_matrix))):
        
        #find row index where user_num_id is i
        a = df.index[df['user_num_id'] == i].tolist()
        
        #loop through the busienss id that the user i reviewed for in offvalid set 
        for item_idx in  item_idx_matrix[i]:
            
            #get the row index for reviews for business that the user liked in the train set
            b = df.index[df['business_num_id'] == item_idx].tolist()
            
            #Find the index for which this user liked, one user only rate a business once
            idx_to_add = list(set(a).intersection(b))
            
            if idx_to_add not in lst:
                lst.extend(idx_to_add)
    return lst

## Preprocess using Term Frequency - CounterVectorizer

In [14]:
#Stemming and Lemmatisation
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
# Get corpus and CountVector
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('wordnet')
nltk.download('stopwords')
lem = WordNetLemmatizer()
stem = PorterStemmer()
stop_words = set(stopwords.words("english"))
new_words = ['not_the']
stop_words = stop_words.union(new_words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shenti10\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shenti10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
from tqdm import tqdm

#Should 'because' added?
def preprocess(df, reset_list = [',','.','?',';','however','but']):
    corpus = []
    for i in tqdm(range(df.shape[0])):
        text = df['review_text'][i]
        change_flg = 0
        #Convert to lowercase
        text = text.lower()
        
        ##Convert to list from string, loop through the review text
        text = text.split()
        
        #any sentence that encounters a not, the folloing words will become not phrase until hit the sentence end
        for j in range(len(text)):
            #Make the not_ hack
            if text[j] == 'not':
                change_flg = 1
#                 print 'changes is made after ', i
                continue
            #if was 1 was round and not hit a 'not' in this round
            if change_flg == 1 and any(reset in text[j] for reset in reset_list):
                text[j] = 'not_' + text[j]
                change_flg = 0
#                 print 'reset at ', i
            if change_flg == 1:
                text[j] = 'not_' + text[j]
        
        #Convert back to string
        text = " ".join(text)
        
        #Remove punctuations
#       text = re.sub('[^a-zA-Z]', ' ', text)
        
        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
        
        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)
        
        ##Convert to list from string
        text = text.split()
        
        ##Stemming
        ps=PorterStemmer()
        
        #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in  
                stop_words] 
        text = " ".join(text)
        corpus.append(text)
    return corpus

## All Model

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, X, row_name = 'business_num_id', binary = True, shape = (121994,6000)):
    """
    get the item-keyphrase matrix
    """
    rows = []
    cols = []
    vals = []
    #For each review history
    for i in tqdm(range(X.shape[0])):
        #Get the array of frequencies for document/review i 
        arr = X[i].toarray() 
        nonzero_element = arr.nonzero()[1]  # Get nonzero element in each line, keyphrase that appears index 
        length_of_nonzero = len(nonzero_element) #number of important keyphrase that appears
        
        # df[row_name][i] is the item idex
        #Get a list row index that indicates the document/review
        rows.extend(np.array([df[row_name][i]]*length_of_nonzero)) ## Item index
        #print(rows)
        
        #Get a list of column index indicating the key phrase that appears in i document/review
        cols.extend(nonzero_element) ## Keyword Index
        if binary:
            #Create a bunch of 1s
            vals.extend(np.array([1]*length_of_nonzero))
        else:
            #If not binary 
            vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)


#Get a UI matrix if it's not item_similarity based or else IU
def predict(matrix_train, k, similarity, item_similarity_en = False):
    prediction_scores = []
    
    #inverse to IU matrix
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    #for each user or item, depends UI or IU 
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores for all items
        #Get prediction/similarity score for each user 1*num or user or num of items
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        #Decending accoding to similarity score, select top k
        similar_users = vector_u.argsort()[::-1][1:k+1]
        
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        
        #similar_users_weights_sum = np.sum(similar_users_weights)
        #print(similar_users_weights.shape)
        #shape: num of res * k
        similar_users_ratings = matrix_train[similar_users].toarray()
              
        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
        #print(prediction_scores_u)
        
        
        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
        
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    return res


#Preidction score is UI or IU?
def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    #for each user
    for user_index in tqdm(range(matrix_Train.shape[0])):
        
        #take the prediction scores for user 1 * num res
        vector_u = prediction_score[user_index]
        
        #The restuarant the user rated
        vector_train = matrix_Train[user_index]
        
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)

#Find prediction for one specific existing user 
def prediction_modified(prediction_score, matrix_Train, user_id, topK = 50):
    prediction = []

    #for each user
    for user_index in tqdm(range(matrix_Train.shape[0])):
        
        #take the prediction scores for user 1 * num res
        vector_u = prediction_score[user_index]
        
        #The restuarant the user rated
        vector_train = matrix_Train[user_index]
        
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine_modified(vector_u, vector_train)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return prediction[user_id]

#One hot encoding for new users, passed in 1 * number items prediction score vector
def prediction_oneHotEncode(prediction_score, initial_record):
    prediction = []

    #for each user
    #for user_index in tqdm(range(matrix_Train.shape[0])):
        
    #take the prediction scores for user 1 * num res
    vector_u =prediction_score

    #The restuarant the user rated 1 * num res
    vector_train = initial_record

    if len(vector_train.nonzero()[0]) > 0:
        vector_predict = sub_routine_modified(vector_u, vector_train)
    else:
        vector_predict = np.zeros(50, dtype=np.float32)

    prediction.append(vector_predict)

    return prediction


#topK: the number of restuarants we are suggesting 
#if vector_train has number, then the user has visited
def sub_routine(vector_u, vector_train, topK=500):

    #index where non-zero
    train_index = vector_train.nonzero()[1]
    
    vector_u = vector_u
    
    #get topk + num rated res prediction score descending, top index 
    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    
    #sort top prediction score index in range topK+len(train_index) into vector_u`
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    
    #deleted the rated res from the topk+train_index prediction score vector for user u 
    #Delete the user rated res index from the topk+numRated index
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    #so we only include the top K prediction score here
    return vector_u[:topK]

def sub_routine_modified(vector_u, vector_train):

    #index where non-zero
    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    #get topk + num rated res prediction score descending, top index 
    candidate_index = np.argpartition(-vector_u, -1)
    
    #sort top prediction score index in range topK+len(train_index) into vector_u`
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    
    #deleted the rated res from the topk+train_index prediction score vector for user u 
    #Delete the user rated res index from the topk+numRated index
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    #so we only include the top K prediction score here
    return vector_u

## Evaluation

In [17]:
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)


def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        #"R-Precision": r_precision,
        #"NDCG": ndcg,
        #"Clicks": click
    }

    local_metrics = {
        #"Precision": precisionk,
        #"Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))
        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = round(results[name],4)
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (round((np.average(results[name])),4),
                                                              round((1.96*np.std(results[name])/np.sqrt(num_users)),4))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))
    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = round(results[name],4)
    else:
        for name in global_metric_names:
            results_summary[name] = (round(np.average(results[name]),4), round((1.96*np.std(results[name])/np.sqrt(num_users)),4))
    output.update(results_summary)

    return output

In [18]:
#Pass in a vector of predicted restaurant for this specific user
#We specify the index of the recommended restaurant we want to display 
def displayRestaurantInfo(RecommendItem_Index, user_item_predictionMatrix,df,ID_dictionary):
    #RecommendItem_Index: 0,1,2,3.. changes as user critiques the restaurant itself
    print('------------------------------------------------------')
    recommend_item = user_item_predictionMatrix[RecommendItem_Index]
    print('Business_num_id: ', recommend_item)
    print('Restaurant name:', df[df['business_num_id'] == recommend_item].name.unique()[0],\
      '\nCuisine Type: ', df[df['business_num_id'] == recommend_item].categories.unique()[0],\
      '\nPrice:', df[df['business_num_id'] == recommend_item].price.unique()[0],\
      '\nRating:', df[df['business_num_id'] == recommend_item].business_stars.unique()[0],\
      '\nDistance:', ID_dictionary[recommend_item], 'km'
     )
    print('------------------------------------------------------')

#### Get original dataframe out of the review datastet

In [19]:
df = get_yelp_df(path ='', filename=reviewJsonToronto, sampling= True)

In [20]:
df.shape[0]

205968

### Investigating the cities

In [22]:
df.head(2)

,Day,Month,Unnamed: 0.1,Unnamed: 0_x,Unnamed: 0_y,Updated,Year,alias,business_id,business_stars,...,review_text,transactions,ufc,url,user_id,user_loc,vote_count,business_num_id,user_num_id,timestamp
0,23,8,6,6,21653,False,2016,happy-lemon-markham,Xo1LNzhnwE-ilqsM3ybs9Q,3.5,...,I ordered the lemon mango slush and the lemon ...,[],"[1, 1, 1]",https://www.yelp.com/biz/happy-lemon-markham?a...,zsJFjhBQEFQ6gJ7BsNM_Ug,"Toronto, Canada",1.0,2682,6984,1.471925e+09
1,2,10,7,7,21653,False,2016,happy-lemon-markham,Xo1LNzhnwE-ilqsM3ybs9Q,3.5,...,"Came here on a Sunday afternoon, it wasn't bus...",[],"[1, 0, 0]",https://www.yelp.com/biz/happy-lemon-markham?a...,P7YuMh74-I2cDq7oU8frww,"York Regional Municipality, Canada",1.0,2682,2845,1.475381e+09


#### Get rating-UI matrix and timestepm-UI matrix

In [23]:
rating_matrix, timestamp_matrix , I_C_matrix, IC_dictionary = get_rating_timestamp_matrix(df)

# get ratingWuserAvg_matrix
rating_array = rating_matrix.toarray()
user_average_array = rating_array.sum(axis = 1)/np.count_nonzero(rating_array,axis = 1)
init_UI = np.zeros(rating_array.shape)
init_UI[rating_array.nonzero()] = 1

#Creating rating with user average array array
for i in range(user_average_array.shape[0]):
    init_UI[i] = init_UI[i] * (user_average_array[i]-0.001) 
user_average_array = init_UI
ratingWuserAvg_array = rating_array - user_average_array
ratingWuserAvg_matrix=sparse.csr_matrix(ratingWuserAvg_array)

In [24]:
IC_dictionary

{'Food Trucks': 90,
 'Mexican': 137,
 'Sandwiches': 172,
 'French': 91,
 'Acai Bowls': 0,
 'Karaoke': 123,
 'Cinema': 51,
 'Diners': 69,
 'Internet Cafes': 116,
 'Team Building Activities': 204,
 'Korean': 126,
 'Reunion': 168,
 'Noodles': 146,
 'Salad': 170,
 'Patisserie/Cake Shop': 151,
 'Supper Clubs': 196,
 'Burgers': 33,
 'Buffets': 32,
 'Brewpubs': 29,
 'Ramen': 166,
 'American (Traditional)': 3,
 'Himalayan/Nepalese': 104,
 'Steakhouses': 193,
 'Pets': 157,
 'Restaurants': 167,
 'Arabian': 5,
 'Bistros': 23,
 'Greek': 97,
 'Poutineries': 163,
 'Sports Clubs': 191,
 'Delicatessen': 65,
 'Social Clubs': 182,
 'Pizza': 158,
 'Lounges': 132,
 'Strip Clubs': 195,
 'Caterers': 42,
 'Laotian': 128,
 'Tiki Bars': 207,
 'Cantonese': 40,
 'Food Stands': 88,
 'Tea Rooms': 203,
 'Tobacco Shops': 208,
 'Street Vendors': 194,
 'Whiskey Bars': 219,
 'Irish': 117,
 'Austrian': 12,
 'Food Tours': 89,
 'Pet Stores': 156,
 'Southern': 186,
 'Vietnamese': 216,
 'Belgian': 22,
 'Waffles': 217,
 'Hot

#### Split to get rtrain-UI matrix and valid and test.. item_index_matrix_train

In [25]:
rtrain_implicit, rvalid_implicit, rtest_implicit, rtrain_userAvg_implicit, rvalid_userAvg_implicit, rtest_userAvg_implicit, nonzero_index, rtime, item_idx_matrix_train_implicit,item_idx_matrix_valid_implicit, item_idx_matrix_test_implicit = time_ordered_splitModified(rating_matrix=rating_matrix, ratingWuserAvg_matrix=ratingWuserAvg_matrix, timestamp_matrix=timestamp_matrix,
                                                                     ratio=[0.5,0.2,0.3],
                                                                     implicit=True,
                                                                     remove_empty=False, threshold=3,sampling=False, 
                                                                     sampling_ratio=0.1, trainSampling=0.95)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  del sys.path[0]
100%|████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:01<00:00, 4991.91it/s]


In [26]:
rtrain, rvalid, rtest, rtrain_userAvg, rvalid_userAvg, rtest_userAvg, nonzero_index, rtime, item_idx_matrix_train,item_idx_matrix_valid, item_idx_matrix_test = time_ordered_splitModified(rating_matrix=rating_matrix, ratingWuserAvg_matrix=ratingWuserAvg_matrix, timestamp_matrix=timestamp_matrix,
                                                                     ratio=[0.5,0.2,0.3],
                                                                     implicit=False,
                                                                     remove_empty=False, threshold=3,
                                                                     sampling=False, sampling_ratio=0.1, 
                                                                     trainSampling=0.95)  

100%|████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:01<00:00, 3858.59it/s]


### Using entire dataset 

In [27]:
rtrain = rtrain + rvalid + rtest

In [28]:
rtrain_implicit = rtrain_implicit + rvalid_implicit + rtest_implicit

### Using TD-IDF to compute corpus and X (business vs. terms) TfIdfVectorizer

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
corpus = preprocess(df)

100%|████████████████████████████████████████████████████████████████████████| 205968/205968 [01:09<00:00, 2978.93it/s]


In [31]:
#Creating a dictionary to store business: review text
dict_text = {}
for i in range(len(corpus)):
    if df['business_num_id'][i] not in dict_text:
        dict_text[df['business_num_id'][i]] = corpus[i]
    else:
        temp = dict_text[df['business_num_id'][i]]
        temp = temp + corpus[i]
        dict_text[df['business_num_id'][i]] = temp

In [32]:
#Create a list for the review text, where the row dimension = total business ids
list_text = []
for key in range(0,max(list(dict_text.keys()))+1) :
    if key not in dict_text.keys():
        list_text.extend([""])
    else:
        list_text.extend([dict_text[key]])

In [33]:
#Get the X vector, where dimension is #business vs #terms like IK
vectorizer = TfidfVectorizer(max_df=0.9,stop_words=stop_words, max_features=5000, ngram_range=(1,1))
X_cleaned = vectorizer.fit_transform(list_text).toarray()
X_cleaned_sparse = csr_matrix(X_cleaned)

In [34]:
#Check keywords
#print(vectorizer.get_feature_names())
#keywordList = X_cleaned_sparse[50].nonzero()[1]
#for word in keywordList:
#    print(vectorizer.get_feature_names()[word])

## Cross Validation Section below

In [35]:
#Passing in the trained similarity matrx
def individualKNNPrediction (similarityMatrix, predictionMatrix, kRange, validOrTestMatrix, itemBased=False):
    "Declaration for kRange = range(50,120,10)"
    #similarity = train(similarityMatrix)
    MAP10 = {}
    #Loop through the kvalues 
    for kValue in kRange:
        if(itemBased==False):
            user_item_p
            rediction_score = predict(predictionMatrix, kValue, similarityMatrix, item_similarity_en= False)
        else:
            user_item_prediction_score = predict(predictionMatrix, kValue, similarityMatrix, item_similarity_en= True)
        user_item_predict = prediction(user_item_prediction_score, 50, predictionMatrix)
        user_item_res = evaluate(user_item_predict, validOrTestMatrix)
        
        
        MAP10[kValue] = user_item_res.get('MAP@10')
        
    return MAP10

def get_UC_Matrix(IC_Matrix,rtrain_implicit):
    U_C_matrix_explicit = rtrain_implicit*IC_Matrix
    U_C_matrix_implicit = getImplicitMatrix(U_C_matrix_explicit,3)
    return U_C_matrix_explicit,U_C_matrix_implicit

def getImplicitMatrix(sparseMatrix, threashold=0):
    temp_matrix = sparse.csr_matrix(sparseMatrix.shape)
    temp_matrix[(sparseMatrix > threashold).nonzero()] = 1
    return temp_matrix

In [36]:
#Passing in the trained similarity matrx
def KNNPrediction (similarityMatrix, predictionMatrix, kValue, validOrTestMatrix, itemBased=False):

    if(itemBased==False):
        user_item_prediction_score = predict(predictionMatrix, kValue, similarityMatrix, item_similarity_en= False)
    else:
        user_item_prediction_score = predict(predictionMatrix, kValue, similarityMatrix, item_similarity_en= True)
    user_item_predict = prediction(user_item_prediction_score, 50, predictionMatrix)
    user_item_res = evaluate(user_item_predict, validOrTestMatrix)

        
    return user_item_res.get('MAP@10')

In [37]:
def saveDictToJson(dictionary, fileName, trainOrTest='train'):
    json_fileName = "{:s}.json".format(fileName)
    if(trainOrTest == 'train'):
        json.dump(dictionary, open("crossValidation\\trainPerformance\\"+json_fileName, 'w') )
    else:
        json.dump(dictionary, open("crossValidation\\testPerformance\\"+json_fileName, 'w') )
    

def loadDict(fileName, trainOrTest='train'):
    json_fileName = "{:s}.json".format(fileName)
    # Read data from file:
    if(trainOrTest == 'train'):
        dataDict = json.load( open("crossValidation\\trainPerformance\\"+json_fileName))
    else:
        dataDict = json.load( open("crossValidation\\testPerformance\\"+json_fileName))
    return dataDict

## Item based recommend & critique INTERNAL TESTING FOR CHECKING API

In [93]:
#intersections
yonge_and_finch = Point("43.779824, -79.415665")
bloor_and_bathurst = Point("43.665194,-79.411208")
queen_and_spadina = Point("43.648772,-79.396259")
bloor_and_yonge = Point("43.670409,-79.386814")
dundas_and_yonge = Point("43.6561,-79.3802")
spadina_and_dundas = Point("43.653004,-79.398082")

#Set intersection for test case:
intersection = spadina_and_dundas

In [94]:
#IK TF-IDF
IK_MATRIX = X_cleaned_sparse
IK_similarity = train(IK_MATRIX)
IC_similarity = train(I_C_matrix)

In [106]:
#Get IP dictionary
#IP_df is the dataframe, IP_dictionary maps business_num_id with price range from 1-4
IP_df, IP_dictionary = get_IP_matrix_dictionary(df, IK_similarity)
IS_dictionary = get_IS_dictionary(df)
categoryList = list(IC_dictionary.keys())
#ID_dictionary needed to be set in later on method
ID_dictionary = get_ID_dictionary(df,list(set(df['business_num_id'])),intersection)

100%|█████████████████████████████████████████████████████████████████████████████| 4996/4996 [00:06<00:00, 750.36it/s]


### 1.Get user initial restaurant set up

In [97]:
pickList = [4331,2970,2216,3628,1844,4972,926,3286,3356,2949,3569,3792,2766,\
     3770,2931,1789,1908,4008,1658,3372,2442,4715,2188,1813,4111,4016,\
    1058,2384, 371,4227]
business_id_list = [df[df['business_num_id']==item].business_id.unique()[0] for item in pickList]
pickBusinessDict = {}
for item in pickList:
    business_name = df[df['business_num_id']==item].name.unique()[0]
    business_id = df[df['business_num_id']==item].business_id.unique()[0]
    pickBusinessDict[business_id] = business_name

In [98]:
print('pick from the following business ids')
pickBusinessDict

pick from the following business ids


{'r_BrIgzYcwo1NAuG9dLbpg': 'Pai Northern Thai Kitchen',
 'aLcFhMe6DDJ430zelCpd2A': 'Khao San Road',
 'RtUvSWO_UZ8V3Wpj0n077w': 'KINKA IZAKAYA ORIGINAL',
 'iGEvDk6hsizigmXhDKs2Vg': 'Seven Lives Tacos Y Mariscos',
 'N93EYZy9R0sdlEvubu94ig': 'Banh Mi Boys',
 'zgQHtqX0gqMw1nlBZl2VnQ': 'Momofuku Noodle Bar',
 'B70iTJjcPkuYn8ouUewWgw': 'Gusto 101',
 'e41TP5cXZqSrz50xCBJqZw': 'Insomnia Restaurant & Lounge',
 'f5O7v_X_jCg2itqacRfxhg': 'Sansotei Ramen',
 'a8pmtlVKf7NiSLI-4KejIw': 'El Catrin Destileria',
 'h_4dPV9M9aYaBliH1Eoeeg': 'Wvrst',
 'kOFDVcnj-8fd3doIpCQ06A': "Mildred's Temple Kitchen",
 'Yl2TN9c23ZGLUBSD9ks5Uw': 'Byblos',
 'k6zmSLmYAquCpJGKNnTgSQ': 'The Stockyards',
 '_xAJZOKBMPOe47p1MphB2w': 'Fishman Lobster Clubhouse Restaurant',
 'MS-hfug4QDXqb_Mws3qlzA': 'Grand Electric',
 'O1TvPrgkK2bUo5O5aSZ7lw': 'Rol San',
 'nBl_4gw5ecGzNkHyzfii8g': 'Terroni',
 'KVpvE6pOPM9KMvak4HFsNg': 'Pizzeria Libretto',
 'fGurvC5BdOfd5MIuLUQYVA': 'Dumpling House Restaurant',
 'UxWH8zRYIBgs6Q2oykvRdw': 'Cluny B

### 2.Initialize, input user preference

In [99]:
#Initialize user preference dictionary 'business_id': '5'
UserInitialResponse = {}
businessIdInput = ''
while businessIdInput.lower() != 'stop': 
    businessIdInput = input("Input business id that you like ").strip()
    print(businessIdInput)
    if businessIdInput== 'stop':
        break;
    elif businessIdInput not in pickBusinessDict.keys():
        print('invalid business id, RE-ENTER')
    else:
        UserInitialResponse[businessIdInput] = 5
print(UserInitialResponse)

Input business id that you like r_BrIgzYcwo1NAuG9dLbpg
r_BrIgzYcwo1NAuG9dLbpg
Input business id that you like aLcFhMe6DDJ430zelCpd2A
aLcFhMe6DDJ430zelCpd2A
Input business id that you like stop
stop
{'r_BrIgzYcwo1NAuG9dLbpg': 5, 'aLcFhMe6DDJ430zelCpd2A': 5}


In [100]:
#Preprocess User Preference Input, transfers the input json into csr sparse matrix, 1 row
#Get row lis
rowList = [0] * len(UserInitialResponse)
#Get col list
colList = []
#Get data list
dataList = []

for resBusinessId, rating in UserInitialResponse.items():
    print(resBusinessId, rating)
    #Append the bus_num_id as column values
    colList.append(df[df['business_id']==resBusinessId].business_num_id.unique()[0])
    dataList.append(rating)

rows = np.array(rowList)
cols = np.array(colList)
data = np.array(dataList)
print(rows, cols,data)

#Get explicit data, dimension: 1 * # of items
userSetUpVector = csr_matrix((data, (rows, cols)), shape=(1, IK_similarity.shape[1]))
print(userSetUpVector)

r_BrIgzYcwo1NAuG9dLbpg 5
aLcFhMe6DDJ430zelCpd2A 5
[0 0] [4331 2970] [5 5]
  (0, 2970)	5
  (0, 4331)	5


In [102]:
user_item_prediction_score = predict(userSetUpVector, 110, IK_similarity, item_similarity_en= True)
#reduce dimension 
user_item_prediction_score = user_item_prediction_score[0]

100%|████████████████████████████████████████████████████████████████████████████| 4996/4996 [00:02<00:00, 1688.35it/s]


In [107]:
user_item_predict = prediction_oneHotEncode(user_item_prediction_score, userSetUpVector)
test_user_item_predict = list(user_item_predict[0])
test_user_item_predict[:5]

recBusinessJson = constructRestaurantDict(recommendIndex, test_user_item_predict\
                                                  ,df,ID_dictionary)
print('Check if this is the same with the API return',recBusinessJson)
    

{'business_id': '4664', 'name': 'Sukho Thai', 'cuisine': 'Thai', 'price': '$$', 'rating': '3.5', 'distance': '2.0km', 'addText': ''}


## Check if item and categories are matched

In [125]:
# def checkIfExhaustedList (current_user_item_predict, original_user_item_predict, critique_Res_list):
#     message = ''
#     if len(current_user_item_predict) == 0:
#         print('Exhausted the recommendation list, fall back to initial preference')
#         current_user_item_predict = [item for item in original_user_item_predict if item not in critique_Res_list]
#         message = 'exhausted critiquing list, fall back'
#     return current_user_item_predict, message

def updateList (listTobeUpdated, listUsedToUpdate):
    tempList = listTobeUpdated.copy()
    tempList.append(listUsedToUpdate)
    listUpdate = list(set(tempList))
    
    return listUpdate

def exhaustFallBack(tempRecommendList):
    ifExhaust = False
    #print(critiqueDictionary['critiqued_Res_AccrdName_list'])
    #Exhausted recommendation list
    if(len(tempRecommendList) == 0):
        ifExhaust = True
        print('Exhausted recommendation list, no matching restaurant after current critique')
        
        #update critiqued restaurant list according to name
        #critiqueDictionary['critiqued_Res_AccrdName_list'] = updateList(critiqueDictionary['critiqued_Res_AccrdName_list'], \
        #                                                                critiqueDictionary['current_user_item_predict'][recommendIndex])
        #Update critiqued restaurant total set
        #critiqueDictionary['current_user_item_predict'] = critiqueDictionary['current_user_item_predict'][recommendIndex+1 :]
    #print(critiqueDictionary['critiqued_Res_AccrdName_list'])
    return ifExhaust

def updateExtendList (listTobeUpdated, listUsedToUpdate):
    tempList = listTobeUpdated.copy()
    tempList.extend(listUsedToUpdate)
    listUpdate = list(set(tempList))
    
    return listUpdate

#This method should generate a dictionary for the recommended item, to be used for API 
def constructRestaurantDict(RecommendItem_Index, user_item_predictionMatrix,df,ID_dictionary, additionalText='', empty=False):
    restaurantInfo = {}
    if empty:
        restaurantInfo['business_id'] = "empty"
        restaurantInfo['name'] = "empty"
        restaurantInfo['cuisine'] = "empty"
        restaurantInfo['price'] = "empty"
        restaurantInfo['rating'] = "empty"
        restaurantInfo['distance'] = "empty"
        #Additional text sent back from the system 
        restaurantInfo['addText'] = "empty"
    else:
        businessId = user_item_predictionMatrix[RecommendItem_Index]
        #print(businessId)
        retrieveDF = df[df['business_num_id'] == businessId]
        restaurantInfo['business_id'] = str(businessId)
        restaurantInfo['name'] = str(retrieveDF.name.unique()[0])
        restaurantInfo['cuisine'] = str(retrieveDF.categories.unique()[0])
        restaurantInfo['price'] = str(retrieveDF.price.unique()[0])
        restaurantInfo['rating'] = str(retrieveDF.business_stars.unique()[0])
        restaurantInfo['distance'] = str(ID_dictionary[businessId]) + 'km'
        #Additional text sent back from the system 
        restaurantInfo['addText'] = additionalText
        
        explanation_dict = df_explanation[df_explanation["business_id"] == str(retrieveDF.business_id.unique()[0])].explanation.values[0]
        exp_list = []
        explanation_str = ''
        for (k,v) in explanation_dict.items():
            explanation_str+=v
            exp_list.append(v)
        explanation_str = exp_list[0] + ", " + exp_list[1] + " and "+ exp_list[2]

        restaurantInfo['explanation'] = explanation_str
    
    return restaurantInfo

                
#Enter critique res list and critique res according to name if fall back to square one 
#deleted critique_Res_list=[], 
def fallBackSetup(critiqued_Res_AccrdName_list=[], critiqueDistance=2.0):
    critique_distance = critiqueDistance
    wanted_Category_index = -1
    variableDictionary['critique_distance'] = critique_distance
    variableDictionary['wanted_Category_index'] = wanted_Category_index
    
    #initialize critique dictionary 
    for listName in critiqueDictionary.keys():
        critiqueDictionary[listName] = []
        
    critiqueDictionary['critique_Res_list'] = [key for (key, value) in ID_dictionary.items() \
                                               if value > variableDictionary['critique_distance']].extend(critiqued_Res_AccrdName_list)
    #critiqueDictionary['critiqued_Res_AccrdName_list'] = critiqued_Res_AccrdName_list
    critiqueDictionary['critiqued_Res_AccrdDistance_list'] = [key for (key, value) in ID_dictionary.items() \
                                                              if value > variableDictionary['critique_distance']]
    critiqueDictionary['current_user_item_predict'] = [item for item in test_user_item_predict if \
                                                       item not in critiqueDictionary['critique_Res_list']]
    
    #Construct recommend business json, insert into dictionary
    recBusinessJson = constructRestaurantDict(recommendIndex, critiqueDictionary['current_user_item_predict'],df,ID_dictionary)
    print(recBusinessJson)
    recommendResDict['recBusinessJson'] = recBusinessJson

In [119]:
pip install flask-restful

Note: you may need to restart the kernel to use updated packages.


In [122]:
#For SSL certificate
pip install -U pyOpenSSL

SyntaxError: invalid syntax (<ipython-input-122-d12770b73b22>, line 2)

In [83]:
def constructUserItemVector(UserInitialResponse):
    #Get row lis
    rowList = [0] * len(UserInitialResponse)
    #Get col list
    colList = []
    #Get data list
    dataList = []

    for resBusinessId, rating in UserInitialResponse.items():
        print(resBusinessId, rating)
        #Append the bus_num_id as column values
        colList.append(df[df['business_id']==resBusinessId].business_num_id.unique()[0])
        dataList.append(rating)

    rows = np.array(rowList)
    cols = np.array(colList)
    data = np.array(dataList)
    print(rows, cols,data)

    #Get explicit data, dimension: 1 * # of items
    userSetUpVector = csr_matrix((data, (rows, cols)), shape=(1, IK_similarity.shape[1]))
    print(userSetUpVector)
    return userSetUpVector

In [65]:
#Resource allows more segreggated 
from flask import Flask, request
from flask_restful import Resource, Api
from sqlalchemy import create_engine
from json import dumps
from flask import jsonify
import json
import requests

In [124]:
#Directly using global variables is fine, but you can only modify dictionary variables

app = Flask(__name__)
#Add resources to be much cleaner
api = Api(app)

#Initialize variable 
#Need to initialize to have only restaurants within 2.0km recommended, restaurants distance above 2.0 will be critiqued
#critique_distance = 2.0

#list of restaurant don't wanted, initialized as restaurants distance <2.0km
#critique_Res_list = [key for (key, value) in ID_dictionary.items() if value > critique_distance] 
#critique_Price_list = []
#ciritiqued_Rating_list = []
#list of categories don't wanted - Accumulated 
#critique_Cat_list = []

#Used to fall back, when user inputs are conflicted, modified during the critiquing process
#critiqued_Res_AccrdName_list= []
#critiqued_Res_AccrdCuisine_list = []
#critiqued_Res_AccrdPrice_list = []
#critiqued_Res_AccrdStar_list = []
#critiqued_Res_AccrdDistance_list = [key for (key, value) in ID_dictionary.items() if value > critique_distance] 

#Categories that explicitly wanted, ONLY 1 FOR NOW
#wanted_Category_index = -1

#Recommendation info
recommendIndex = 0

#To be initialized
ID_dictionary = {}

#Initial user item vector for critiquing process 
#current_user_item_predict = [item for item in test_user_item_predict if item not in critique_Res_list]

#Construct initial recommendation
#recBusinessJson = constructRestaurantDict(recommendIndex, current_user_item_predict,df,ID_dictionary)
#print(recBusinessJson)

#Following dictionaries are for retrieval 

#Construct constant variables dictionary
variableDictionary = {
    'critique_distance' : 2.0,
    'wanted_Category_index': -1,
    'max_distance': 169
}

#A dictionary that stores all the lists related with the critiquing process
critiqueDictionary = {
    'critique_Res_list': [],
    'critique_Price_list':[],
    'critique_Cat_list': [],
    'ciritiqued_Rating_list':[],
    'critiqued_Res_AccrdStar_list': [],
    'critiqued_Res_AccrdCuisine_list':[],
    'critiqued_Res_AccrdPrice_list':[],
    'critiqued_Res_AccrdName_list': [],
    'critiqued_Res_AccrdDistance_list':[],
    'test_user_item_predict': [],
    'current_user_item_predict': []
    }

#Storing the current recommended restaurant
recommendResDict = {
        'recBusinessJson': None #recBusinessJson, to be initialized
}


class ClearUp(Resource):
    #Request for resetting user preferences 
    def get(self):
        fallBackSetup(critiqueDictionary['critiqued_Res_AccrdName_list'], critiqueDistance=2.1)
        return {'You are Resetting the user set up', 'Not retrieving anything for you'}

class Initialize(Resource):
    def get(self):
        return {'You are calling a GET for set up:': 'Not retrieving anything for you'}
    
    #Put user initial preferences to initialize the initial user recommendation 
    #Produces test_user_item_predict, computes the recommendation results for the new user 
    #request json {business id: rating}
    def put(self):
        request_json  = request.get_json()
        print(request_json)
        
        #Construct the user item rating vector 
        userSetUpVector = constructUserItemVector(request_json)
        #Get item recommendation for user 
        user_item_prediction_score = predict(userSetUpVector, 110, IK_similarity, item_similarity_en= True)[0]
        user_item_predict = prediction_oneHotEncode(user_item_prediction_score, userSetUpVector)
        #Get the user item prediction, put into dictionary 
        test_user_item_predict = list(user_item_predict[0])
        critiqueDictionary['test_user_item_predict'] = test_user_item_predict.copy()
        
        return {'You are calling a PUT for set up, the user initialization you sent:': request_json}
    
    #Post intersection {'latitude': latitude, 'longitude': longitude}
    def post(self):
        request_json  = request.get_json()
        print(request_json)
        latitude = str(request_json.get('latitude', 'none'))
        longitude = str(request_json.get('longitude', 'none'))
        intersectionString = latitude + ',' + longitude
        intersection = Point(intersectionString)

        #Create ID_dictionary 
        ID_dictionary = get_ID_dictionary(df,list(set(df['business_num_id'])),intersection) 

        variableDictionary['maxDistance'] = max(ID_dictionary, key=ID_dictionary.get)

        #create initial critique restaurant list
        critique_Res_list = [key for (key, value) in ID_dictionary.items() if value > variableDictionary['critique_distance']] 
        critiqued_Res_AccrdDistance_list = critique_Res_list.copy()
        critiqueDictionary['critique_Res_list'] = critique_Res_list.copy()
        critiqueDictionary['critiqued_Res_AccrdDistance_list'] = critiqued_Res_AccrdDistance_list.copy()
        
        #Creates the rest -- waiting for put method to initialize user 
        current_user_item_predict = [item for item in critiqueDictionary['test_user_item_predict']\
                                     if item not in critiqueDictionary['critique_Res_list']]
        critiqueDictionary['current_user_item_predict'] = current_user_item_predict.copy()
        
        recBusinessJson = constructRestaurantDict(recommendIndex, critiqueDictionary['current_user_item_predict']\
                                                  ,df,ID_dictionary)
        print(recBusinessJson)
        recommendResDict['recBusinessJson'] = recBusinessJson

        return {'You are calling a POST for set up, intersection point:': intersectionString}
    
class Business(Resource):

    def __init__(self):
        self.critique_Price_list = []
        self.countVar = 0
    
    def get(self):
        #return {'business name': data[0]['name']} # Fetches first column that is Employee ID
        return recommendResDict['recBusinessJson']
    
    def post(self):
        some_json  = request.get_json()
        #put json object for the critiquing process 
        #Return newly computed object 
        return {'You sent': some_json}, 201
    def put(self):
        #Get the request json
        request_json  = request.get_json()
        print('Input data:', request_json)
        
        #put json object for the critiquing process 
        critique_feature = request_json.get('feature', 'none')
        critique_posNeg = request_json.get('positiveOrNegative', 'none')
        critique_value = request_json.get('critiqueValue', 'none')
        if isinstance(critique_value, list):
            critique_value = critique_value[0]
        
        #Error Handling - Shouldn't trigger if json format passed in properly 
        if critique_feature == 'none':
            print('Critique Feature cannot be recognized')
        if critique_posNeg == 'none':
            print('Critique positive or negative category cannot be recognized')
        
        #Once user send this request, critique current item 
        #tempCritiqueRestList = critiqueDictionary['critique_Res_list'].copy()
        #tempCritiqueRestList.append(critiqueDictionary['current_user_item_predict'][recommendIndex])
        
        #Put back into the dictionary 
        critiqueDictionary['critique_Res_list'].append(critiqueDictionary['current_user_item_predict'][recommendIndex])
        critiqueDictionary['critiqued_Res_AccrdName_list'].append(critiqueDictionary['current_user_item_predict'][recommendIndex])
        critiqueDictionary['current_user_item_predict'] = critiqueDictionary['current_user_item_predict'][recommendIndex+1 :]
        
        #initialize additional text to be passed back
        additionalText = ''
        #temporary updated recommendation list
        tempRecommendList = []
        #Scenario 1 - Critique Restaurant name 
        if 'name' in critique_feature:
            
            print("Saving critiqued item: ", critiqueDictionary['current_user_item_predict'][recommendIndex])      
            additionalText = "Saving critiqued item: " + str(critiqueDictionary['current_user_item_predict'][recommendIndex])
            
            #update critiqued restaurant list according to name
            #critiqueDictionary['critiqued_Res_AccrdName_list'] = updateList(critiqueDictionary['critiqued_Res_AccrdName_list'], \
            #                                                                critiqueDictionary['current_user_item_predict'][recommendIndex])
            
            
        #Scenario 2 - Negatively critique restaurant features 
        if ('cuisine' in critique_feature) and ('negative' in critique_posNeg):
            
            #Get current recommended restaurant cuisine type
            currentCuisineType = recommendResDict['recBusinessJson']['cuisine'].split(',')
            
            #Find the correct category name the user want to critique in current recommended item categories
            critiqueValue = [cuisine for cuisine in currentCuisineType if critique_value.strip().lower() in cuisine.lower()][0]
            
            #Retrieve critiquing category index
            critiquied_Cat_Index = IC_dictionary[critiqueValue]
            print('Saving negatively critiqued cuisine type:', critique_value, ', cuisine index: ', critiquied_Cat_Index)

            #Save the newly critiqued cuisine type
            critiqueDictionary['critique_Cat_list'] = critiqueDictionary['critique_Cat_list'].append(critiquied_Cat_Index)
            
            #Handling extreme case - critique categories previously requested 
            if critiquied_Cat_Index == variableDictionary['wanted_Category_index']:
                #Reset wanted category index
                variableDictionary['wanted_Category_index'] = -1
                print('You are critiquing a cuisine type you previously requested\n Fall back to initial preference')
                additionalText = additionalText + '.You are critiquing a cuisine type you previously requested\n Fall back to initial preference'
                
                #Restting the prediction list -.-? 
                critiqueDictionary['current_user_item_predict'] = \
                                        [item for item in critiqueDictionary['test_user_item_predict'] if \
                                         item not in critique_Res_list]

            #Find list of restaurants to filter out 
            critiquedItemsList = list(I_C_matrix.getcol(critiquied_Cat_Index).nonzero()[0])
            
            #Updating the critiqued items to list
            print('Saving critiqued items at index: ', critiquedItemsList[:5], '...')
            templist = critiqueDictionary['critique_Res_list'].copy()
            templist.extend(list(critiquedItemsList))
            
            #temporary updated recommend list 
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict']\
                                          if item not in templist]
            
            #Check if exhausted list, if so clear out all restaurants 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, falling back to initial preference'
                print('--FALLING BACK TO INITIAL PREFERENCE--')
                fallBackSetup(critique_Res_list=critiqueDictionary['critique_Res_list'], \
                              critiqued_Res_AccrdName_list=critiqueDictionary['critiqued_Res_AccrdName_list'], \
                              critiqueDistance=2.0)
            else:
                #Update critique restaurant list 
                critiqueDictionary['critique_Res_list'] = list(set(templist))

                #Update the list of restaurants critiqued by cuisin type
                templist = critiqueDictionary.get('critiqued_Res_AccrdCuisine_list',[]).copy()
                templist.extend(critiquedItemsList)
                critiqueDictionary['critiqued_Res_AccrdCuisine_list'] = list(set(templist))

                #Filter out critiqued items, sequence must remain the same, update current valid set 
                critiqueDictionary['current_user_item_predict'] = tempRecommendList.copy()

            #Handle case where run out of items! - Fall Back!
            #critiqueDictionary['current_user_item_predict'], message = checkIfExhaustedList(critiqueDictionary['current_user_item_predict'], \
            #                                                                       critiqueDictionary['test_user_item_predict'], \
            #                                                                       critiqueDictionary['critique_Res_list'])
            #additionalText = additionalText + '. ' + message
        
        #Scenario 3 - Positively critique restaurant cuisine type 
        if 'cuisine' in critique_feature and 'positive' in critique_posNeg:
            #Find the correct category within all the categories list - assuming exact word typed in 
            """Add ERROR HANDLING HERE"""
            positiveCritiquiCategory = [cuisine for cuisine in categoryList if \
                                        critique_value.strip().lower() in cuisine.lower()][0]

            #Retrieve cuisine type index, 1 number  
            positiveCritiquiedIndex = IC_dictionary[positiveCritiquiCategory]

            #Get the preferred category index, replace existing ones
            variableDictionary['wanted_Category_index'] = positiveCritiquiedIndex

            #Retrieve items/restaurants matching item category 
            matchedResList = list(I_C_matrix.getcol(positiveCritiquiedIndex).nonzero()[0])
            print('matching list:', matchedResList[:10])
            
            #additionalText = 'positive critique cuisine type' + str(positiveCritiquiCategory) + 'id:' + str(positiveCritiquiedIndex)\
            #                    +'critiquing business: ' +str(matchedResList[:5]) + '...'

            #Check if user have previously critiqued - remove it 
            if positiveCritiquiedIndex in critiqueDictionary['critique_Cat_list']:
                templist = critiqueDictionary['critique_Cat_list']
                templist.remove(positiveCritiquiedIndex)
                critiqueDictionary['critique_Cat_list'] = templist

                #Update critiqued restaurant list according to cuisine type 
                templist = critiqueDictionary['critiqued_Res_AccrdCuisine_list']
                templist.remove(matchedResList)
                critiqueDictionary['critiqued_Res_AccrdCuisine_list'] = templist

                #UPDATE CRITIQUE RESTAURANT LIST 
                #In the matched restuarnt list and not critiqued under other criteria will be withdrawn from this list
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict']\
                                                           if item not in matchedResList or \
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list']+ \
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdStar_list']+ \
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))]
            
            tempRecommendList = [item for item in matchedResList if item not in critiqueDictionary['critique_Res_list']]
            
            #Check if exhausted recommendation list 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            else:    
                #Update current valid set, make sure not in critiqued restaurant set 
                critiqueDictionary['current_user_item_predict'] = tempRecommendList.copy()
            
        #Scenario 4 - Negative critique restuarant price e.g. "I don't want expensive restaurants" OR positive "I want cheaper"
        #Scenario 5 - Positively critique restaurant price, "I want fine dining", wanting more expensive restaurant 
        if 'price' in critique_feature:
            
            #Get current price label range in 1 - 4
            currentPriceLabel = IP_dictionary[critiqueDictionary['current_user_item_predict'][recommendIndex]]
            
            #Get critique_price_list first
            templist = critiqueDictionary.get('critique_Price_list', []).copy()
            
            #Update critiqued price list
            if 'positive' in critique_posNeg and critique_value == 'more expensive':
                templist.extend([price for price in range(1,currentPriceLabel+1,1)])
                
            #Negative or positive, cheaper     
            else:     
                templist.extend([price for price in range(currentPriceLabel,5,1)])
            
            #update critiqued_price_list by puting it back to the dictionary
            critiqueDictionary['critique_Price_list'] = list(set(templist))

            #Check if critiqued all price range, if exhausted, fall back 
            if len(critiqueDictionary['critique_Price_list']) == 4:
                print('You have exhaused the price range option, showing the most', critique_value, 'option')
                #Send feedback to front end
                additionalText = 'You have exhaused the price range option, showing the most '+ critique_value + ' option'
                
                #Clear the previous critiqued restuarants based on price out of critiqued restuarant list 
                #POTENTIAL ISSUE HERE, MAY BE ERASING SOME RESTAURANTS CRITIQUED IN OTHER STEPS ... NEED TO RECHECK
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list'] + \
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdStar_list'] + \
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))]

                #Clear out the restuarants critiquied by price 
                critiqueDictionary['critiqued_Res_AccrdPrice_list'] = []

                #Restore default critiquing price list 
                if critique_value == 'more expensive':
                    critiqueDictionary['critique_Price_list'] = [1,2,3]
                else:
                    critiqueDictionary['critique_Price_list'] = [2,3,4]  
                    
            #printing and sending message to front end, the critiquing price range
            print('Critiqing price at range:', ['$'*label for label in critiqueDictionary['critique_Price_list'] ])
            #additionalText += 'Critiqing price at range:'+str(['$'*label for label in critiqueDictionary['critique_Price_list']])
            
            #Find the list of restaurants to critique 
            listCritiqueRestaurant = [key for (key, value) in IP_dictionary.items() if \
                                       value in critiqueDictionary['critique_Price_list']]
            
            #find temporary updated critique list    
            tempCritiqueList = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRestaurant)
            
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict']\
                                                               if item not in tempCritiqueList]
            
            #CHECK IF EXHAUSTED, if so clear out all restaurants 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            
            else:
                #Update the critiqued restaurant list
                critiqueDictionary['critique_Res_list'] = tempCritiqueList.copy()

                #Record the list of restaurants critiqued so far based on price 
                critiqueDictionary['critiqued_Res_AccrdPrice_list'] = updateExtendList(critiqueDictionary['critiqued_Res_AccrdPrice_list'], \
                                                           listCritiqueRestaurant)
                
                #Updating the critiqued items to list
                print('Saving critiqued items at index: ', listCritiqueRestaurant[:5], '...')

                ##Update current valid set , filter out critiqued items, sequence must remain the same
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict']\
                                                                   if item not in tempCritiqueList]
            
        #Scenario 6 negatively critique rating & positively critique rating: both goes up 
        #"I don't want ratings this low?" "I don't want restuarants with rating below XXX" "I want restaurants with rating above XXX"
        #I will critique the ratings below the current rating or specific rating 
        if 'rating' in critique_feature:
            currentRating = float(recommendResDict['recBusinessJson']['rating'])
            
            if critique_value.lower() == 'current_rating':
                critique_value = currentRating
        
            #Critiquing the restaurants that has ratings equal and below this restaurant 
            critiqueRes = [rating/10 for rating in range(0,int(float(critique_value)*10)+1,5)]
            critiqueDictionary['ciritiqued_Rating_list'] = updateExtendList(critiqueDictionary['ciritiqued_Rating_list'], \
                                                                           critiqueRes)
            #If critiqued all price
            if critiqueDictionary['ciritiqued_Rating_list'] == [price/10 for price in range(0,51,5)]:
                print('exhausted list, no better restaurants, recommending the finest restaurants')
                additionalText += '| Exhausted list, no better restaurants, recommending restaurants beyond current rating'

                #reset critiqued restaurant list 
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))] 

                #Clear out the restuarants critiquied by rating 
                critiqueDictionary['critiqued_Res_AccrdStar_list'] = []
                critiqueDictionary['ciritiqued_Rating_list'] = [rating/10 for rating in range(0,int(currentRating*10+1),5)]

            print('critiquing restaurants at rating rangeing at:',critiqueDictionary['ciritiqued_Rating_list'])
            #additionalText += '| critiquing restaurants at rating rangeing at:'+str(critiqueDictionary['ciritiqued_Rating_list'])

            #Get the list of restaurants to critique
            listCritiqueRes = [key  for (key, value) in IS_dictionary.items() if \
                               value in critiqueDictionary['ciritiqued_Rating_list']]
            
            #CHECK EXHAUSTED - Check to see if user critique exhausts the recommendation list
            #Take a temporary list as updated critiquing list 
            tempCritiqueList = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRes)
            
            #Take a temporary list as updated recommendation list 
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                               if item not in tempCritiqueList]
            #Exhausted recommendation list
            if exhaustFallBack(tempRecommendList):
                additionalText += '| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            else:
                #Updating the critiqued items to list
                print('Saving critiqued items at index: ', listCritiqueRes[:5], '...')

                #Update the critiqued restaurant list
                critiqueDictionary['critique_Res_list'] = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRes)

                #Record those as well
                critiqueDictionary['critiqued_Res_AccrdStar_list'] = updateExtendList(critiqueDictionary['critiqued_Res_AccrdStar_list'], \
                                                                                      listCritiqueRes)
                #Update current valid set 
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                                   if item not in critiqueDictionary['critique_Res_list']]
        
        
        #Scenario #7 Critique on Distance, goes up or down, only say positive critiques. Want closer, further, or specific value 
        if 'distance' in critique_feature:
            
            #Get current distance
            currentDistance = float(recommendResDict['recBusinessJson']['distance'].strip('km'))
            
            #If want further distance restaurants
            if 'positive' in critique_posNeg and 'further' in critique_value:
                print('Critiquing distance <=', currentDistance)
                #New requirements for restaurants to be critiqued
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value <= currentDistance]

            #Closer distance, critique value not empty 
            else:
                if (critique_value) and not 'closer' in critique_value:
                    currentDistance = float(critique_value)

                print('Critiquing distance >=', currentDistance)

                #New requirements for restaurants to be critiqued
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value >= currentDistance]


            #Update critique restaurant list, only keep the ones that are critiqued under other features
            critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                                      (item in list(set(listCritiquedRestaurant+\
                                                                        critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdStar_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdCuisine_list'])))] 

            #Update list of restaurants critiqued by the restuarnt distance, entire replacement
            critiqueDictionary['critiqued_Res_AccrdDistance_list'] = listCritiquedRestaurant

            #Update initial valid restuarnt list
            critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict']\
                                                               if item not in critiqueDictionary['critique_Res_list']]

            #Handle case where critiqued everything, there's no restuarnt that satisfy all the critiques 
            if(len(critiqueDictionary['current_user_item_predict']) == 0):
                print('Exhausted sytem')

                #new_Distance= input('There\'s no restaurant that matches your preference within this area, please input a new larger distance')
                additionalText += 'No restaurants that matches your preference within this area, currently default back to 2.0km'
                new_Distance = 2.0
                
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value > new_Distance]

                #Restore original critiqued restaurants under distance feature 
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    (item in list(set(listCritiquedRestaurant + \
                                                      critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdStar_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdCuisine_list'])))]   
                #New critiquing restuarants
                critiqueDictionary['critiqued_Res_AccrdDistance_list'] = listCritiquedRestaurant

                #Update initial valid restuarnt list
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                                   if item not in critiqueDictionary['critique_Res_list']]
        
        #Full fall back 
        if(len(critiqueDictionary['current_user_item_predict']) ==0):
            fallBackSetup(critique_Res_list=critiqueDictionary['critique_Res_list'], \
                              critiqued_Res_AccrdName_list=critiqueDictionary['critiqued_Res_AccrdName_list'], \
                              critiqueDistance=2.0)
            additionalText += ' Ran out of recommendations at the end of the process, reset, filtered out seen restaurants'
            
            #If still no results
            if len(critiqueDictionary['current_user_item_predict']) ==0:
                fallBackSetup(critique_Res_list=[], \
                              critiqued_Res_AccrdName_list=[], \
                              critiqueDistance=2.0)
                additionalText += ' Clearing EVERYTHING'

        #Construct newly returning item
        return_Json = constructRestaurantDict(recommendIndex, critiqueDictionary['current_user_item_predict'],\
                                              df,ID_dictionary,additionalText)

        #Update restaurant information for the GET method
        recommendResDict['recBusinessJson'] = return_Json.copy()

        #Return newly computed object 
        return {'You sent': request_json, 'Result': return_Json}, 201
            
api.add_resource(Business, '/business') # Route_1
api.add_resource(Initialize, '/initialization') #Route_2
api.add_resource(ClearUp, '/clearup') #Route_3

if __name__ == '__main__':
     app.run(port='5002')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)


{'r_BrIgzYcwo1NAuG9dLbpg': 5, 'aLcFhMe6DDJ430zelCpd2A': 5}
r_BrIgzYcwo1NAuG9dLbpg 5
aLcFhMe6DDJ430zelCpd2A 5
[0 0] [4331 2970] [5 5]
  (0, 2970)	5
  (0, 4331)	5


100%|████████████████████████████████████████████████████████████████████████████| 4996/4996 [00:02<00:00, 1735.75it/s]
127.0.0.1 - - [18/Feb/2020 23:07:22] "PUT /initialization HTTP/1.1" 200 -


{'latitude': '43.653004', 'longitude': '-79.398082'}


100%|█████████████████████████████████████████████████████████████████████████████| 4996/4996 [00:06<00:00, 783.69it/s]


{'business_id': '4664', 'name': 'Sukho Thai', 'cuisine': 'Thai', 'price': '$$', 'rating': '3.5', 'distance': '2.0km', 'addText': ''}


127.0.0.1 - - [18/Feb/2020 23:07:58] "POST /initialization HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2020 23:07:58] "GET /business HTTP/1.1" 200 -


## original API version - backup

In [92]:
#Directly using global variables is fine, but you can only modify dictionary variables

app = Flask(__name__)
#Add resources to be much cleaner
api = Api(app)

#Initialize variable 

#Need to initialize to have only restaurants within 2.0km recommended, restaurants distance above 2.0 will be critiqued
critique_distance = 2.0

#list of restaurant don't wanted, initialized as restaurants distance <2.0km
critique_Res_list = [key for (key, value) in ID_dictionary.items() if value > critique_distance] 
critique_Price_list = []
ciritiqued_Rating_list = []
#list of categories don't wanted - Accumulated 
critique_Cat_list = []

#Used to fall back, when user inputs are conflicted, modified during the critiquing process
critiqued_Res_AccrdName_list= []
critiqued_Res_AccrdCuisine_list = []
critiqued_Res_AccrdPrice_list = []
critiqued_Res_AccrdStar_list = []
critiqued_Res_AccrdDistance_list = [key for (key, value) in ID_dictionary.items() if value > critique_distance] 

#Categories that explicitly wanted, ONLY 1 FOR NOW
wanted_Category_index = -1

#Recommendation info
recommendIndex = 0  

#Initial user item vector for critiquing process 
current_user_item_predict = [item for item in test_user_item_predict if item not in critique_Res_list]

#Construct initial recommendation
recBusinessJson = constructRestaurantDict(recommendIndex, current_user_item_predict,df,ID_dictionary)
print(recBusinessJson)

#Following dictionaries are for retrieval 

#Construct constant variables dictionary
variableDictionary = {
    'critique_distance' : critique_distance,
    'wanted_Category_index': wanted_Category_index
    
}

#A dictionary that stores all the lists related with the critiquing process
critiqueDictionary = {
    'critique_Res_list': critique_Res_list,
    'critique_Price_list':critique_Price_list,
    'critique_Cat_list': critique_Cat_list,
    'ciritiqued_Rating_list':ciritiqued_Rating_list,
    'critiqued_Res_AccrdStar_list': critiqued_Res_AccrdStar_list,
    'critiqued_Res_AccrdCuisine_list':critiqued_Res_AccrdCuisine_list,
    'critiqued_Res_AccrdPrice_list':critiqued_Res_AccrdPrice_list,
    'critiqued_Res_AccrdName_list': critiqued_Res_AccrdName_list,
    'critiqued_Res_AccrdDistance_list':critiqued_Res_AccrdDistance_list,
    'test_user_item_predict': test_user_item_predict,
    'current_user_item_predict': current_user_item_predict
    }

#Storing the current recommended restaurant
recommendResDict = {
        'recBusinessJson': recBusinessJson
}


class Business(Resource):

    def __init__(self):
        self.critique_Price_list = []
        self.countVar = 0
    
    def get(self):
        #return {'business name': data[0]['name']} # Fetches first column that is Employee ID
        return recommendResDict['recBusinessJson']
    
    def post(self):
        some_json  = request.get_json()
        #put json object for the critiquing process 
        #Return newly computed object 
        return {'You sent': some_json}, 201
    
    def put(self):
        #Get the request json
        request_json  = request.get_json()
        print('Input data:', request_json)
        
        #put json object for the critiquing process 
        critique_feature = request_json.get('feature', 'none')
        critique_posNeg = request_json.get('positiveOrNegative', 'none')
        critique_value = request_json.get('critiqueValue', 'none')
        if isinstance(critique_value, list):
            critique_value = critique_value[0]
        
        #Error Handling - Shouldn't trigger if json format passed in properly 
        if critique_feature == 'none':
            print('Critique Feature cannot be recognized')
        if critique_posNeg == 'none':
            print('Critique positive or negative category cannot be recognized')
        
        #Once user send this request, critique current item 
        tempCritiqueRestList = critiqueDictionary['critique_Res_list'].copy()
        tempCritiqueRestList.append(critiqueDictionary['current_user_item_predict'][recommendIndex])
        
        #Put back into the dictionary 
        critiqueDictionary['critique_Res_list'] = list(set(tempCritiqueRestList)).copy()
        critiqueDictionary['critiqued_Res_AccrdName_list'].append(critiqueDictionary['current_user_item_predict'][recommendIndex])
        
        #initialize additional text to be passed back
        additionalText = ''
        #temporary updated recommendation list
        tempRecommendList = []
        
        #Scenario 1 - Critique Restaurant name 
        if 'name' in critique_feature:
            
            print("Saving critiqued item: ", critiqueDictionary['current_user_item_predict'][recommendIndex])      
            additionalText = "Saving critiqued item: " + str(critiqueDictionary['current_user_item_predict'][recommendIndex])
        
            #Update critiqued restaurant total set
            critiqueDictionary['current_user_item_predict'] = critiqueDictionary['current_user_item_predict'][recommendIndex+1 :]
            
            
        #Scenario 2 - Negatively critique restaurant features 
        if ('cuisine' in critique_feature) and ('negative' in critique_posNeg):
            
            #Get current recommended restaurant cuisine type
            currentCuisineType = recommendResDict['recBusinessJson']['cuisine'].split(',')
            
            #Find the correct category name the user want to critique in current recommended item categories
            critiqueValue = [cuisine for cuisine in currentCuisineType if critique_value.strip().lower() in cuisine.lower()][0]
            
            #Retrieve critiquing category index
            critiquied_Cat_Index = IC_dictionary[critiqueValue]
            print('Saving negatively critiqued cuisine type:', critique_value, ', cuisine index: ', critiquied_Cat_Index)

            #Save the newly critiqued cuisine type
            critiqueDictionary['critique_Cat_list'] = critiqueDictionary['critique_Cat_list'].append(critiquied_Cat_Index)
            
            #Handling extreme case - critique categories previously requested 
            if critiquied_Cat_Index == variableDictionary['wanted_Category_index']:
                #Reset wanted category index
                variableDictionary['wanted_Category_index'] = -1
                print('You are critiquing a cuisine type you previously requested\n Fall back to initial preference')
                additionalText = additionalText + '.You are critiquing a cuisine type you previously requested\n Fall back to initial preference'
                
                #Restting the prediction list -.-? 
                critiqueDictionary['current_user_item_predict'] = \
                                        [item for item in critiqueDictionary['test_user_item_predict'] if \
                                         item not in critique_Res_list]

            #Find list of restaurants to filter out 
            critiquedItemsList = list(I_C_matrix.getcol(critiquied_Cat_Index).nonzero()[0])
            
            #Updating the critiqued items to list
            print('Saving critiqued items at index: ', critiquedItemsList[:5], '...')
            templist = critiqueDictionary['critique_Res_list'].copy()
            templist.extend(list(critiquedItemsList))
            
            #temporary updated recommend list 
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict']\
                                          if item not in templist]
            
            #Check if exhausted list, if so clear out all restaurants 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, falling back to initial preference'
                print('--FALLING BACK TO INITIAL PREFERENCE--')
                fallBackSetup(critique_Res_list=critiqueDictionary['critique_Res_list'], \
                              critiqued_Res_AccrdName_list=critiqueDictionary['critiqued_Res_AccrdName_list'], \
                              critiqueDistance=2.0)
            else:
                #Update critique restaurant list 
                critiqueDictionary['critique_Res_list'] = list(set(templist))

                #Update the list of restaurants critiqued by cuisin type
                templist = critiqueDictionary.get('critiqued_Res_AccrdCuisine_list',[]).copy()
                templist.extend(critiquedItemsList)
                critiqueDictionary['critiqued_Res_AccrdCuisine_list'] = list(set(templist))

                #Filter out critiqued items, sequence must remain the same, update current valid set 
                critiqueDictionary['current_user_item_predict'] = tempRecommendList.copy()

            #Handle case where run out of items! - Fall Back!
            #critiqueDictionary['current_user_item_predict'], message = checkIfExhaustedList(critiqueDictionary['current_user_item_predict'], \
            #                                                                       critiqueDictionary['test_user_item_predict'], \
            #                                                                       critiqueDictionary['critique_Res_list'])
            #additionalText = additionalText + '. ' + message
        
        #Scenario 3 - Positively critique restaurant cuisine type 
        if 'cuisine' in critique_feature and 'positive' in critique_posNeg:
            #Find the correct category within all the categories list - assuming exact word typed in 
            """Add ERROR HANDLING HERE"""
            positiveCritiquiCategory = [cuisine for cuisine in categoryList if \
                                        critique_value.strip().lower() in cuisine.lower()][0]

            #Retrieve cuisine type index, 1 number  
            positiveCritiquiedIndex = IC_dictionary[positiveCritiquiCategory]

            #Get the preferred category index, replace existing ones
            variableDictionary['wanted_Category_index'] = positiveCritiquiedIndex

            #Retrieve items/restaurants matching item category 
            matchedResList = list(I_C_matrix.getcol(positiveCritiquiedIndex).nonzero()[0])
            print('matching list:', matchedResList[:10])
            
            #additionalText = 'positive critique cuisine type' + str(positiveCritiquiCategory) + 'id:' + str(positiveCritiquiedIndex)\
            #                    +'critiquing business: ' +str(matchedResList[:5]) + '...'

            #Check if user have previously critiqued - remove it 
            if positiveCritiquiedIndex in critiqueDictionary['critique_Cat_list']:
                templist = critiqueDictionary['critique_Cat_list']
                templist.remove(positiveCritiquiedIndex)
                critiqueDictionary['critique_Cat_list'] = templist

                #Update critiqued restaurant list according to cuisine type 
                templist = critiqueDictionary['critiqued_Res_AccrdCuisine_list']
                templist.remove(matchedResList)
                critiqueDictionary['critiqued_Res_AccrdCuisine_list'] = templist

                #UPDATE CRITIQUE RESTAURANT LIST 
                #In the matched restuarnt list and not critiqued under other criteria will be withdrawn from this list
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict']\
                                                           if item not in matchedResList or \
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list']+ \
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdStar_list']+ \
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))]
            
            tempRecommendList = [item for item in matchedResList if item not in critiqueDictionary['critique_Res_list']]
            
            #Check if exhausted recommendation list 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            else:    
                #Update current valid set, make sure not in critiqued restaurant set 
                critiqueDictionary['current_user_item_predict'] = tempRecommendList.copy()
            
        #Scenario 4 - Negative critique restuarant price e.g. "I don't want expensive restaurants" OR positive "I want cheaper"
        #Scenario 5 - Positively critique restaurant price, "I want fine dining", wanting more expensive restaurant 
        if 'price' in critique_feature:
            
            #Get current price label range in 1 - 4
            currentPriceLabel = IP_dictionary[critiqueDictionary['current_user_item_predict'][recommendIndex]]
            
            #Get critique_price_list first
            templist = critiqueDictionary.get('critique_Price_list', []).copy()
            
            #Update critiqued price list
            if 'positive' in critique_posNeg and critique_value == 'more expensive':
                templist.extend([price for price in range(1,currentPriceLabel+1,1)])
                
            #Negative or positive, cheaper     
            else:     
                templist.extend([price for price in range(currentPriceLabel,5,1)])
            
            #update critiqued_price_list by puting it back to the dictionary
            critiqueDictionary['critique_Price_list'] = list(set(templist))

            #Check if critiqued all price range, if exhausted, fall back 
            if len(critiqueDictionary['critique_Price_list']) == 4:
                print('You have exhaused the price range option, showing the most', critique_value, 'option')
                #Send feedback to front end
                additionalText = 'You have exhaused the price range option, showing the most '+ critique_value + ' option'
                
                #Clear the previous critiqued restuarants based on price out of critiqued restuarant list 
                #POTENTIAL ISSUE HERE, MAY BE ERASING SOME RESTAURANTS CRITIQUED IN OTHER STEPS ... NEED TO RECHECK
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list'] + \
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdStar_list'] + \
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))]

                #Clear out the restuarants critiquied by price 
                critiqueDictionary['critiqued_Res_AccrdPrice_list'] = []

                #Restore default critiquing price list 
                if critique_value == 'more expensive':
                    critiqueDictionary['critique_Price_list'] = [1,2,3]
                else:
                    critiqueDictionary['critique_Price_list'] = [2,3,4]  
                    
            #printing and sending message to front end, the critiquing price range
            print('Critiqing price at range:', ['$'*label for label in critiqueDictionary['critique_Price_list'] ])
            #additionalText += 'Critiqing price at range:'+str(['$'*label for label in critiqueDictionary['critique_Price_list']])
            
            #Find the list of restaurants to critique 
            listCritiqueRestaurant = [key for (key, value) in IP_dictionary.items() if \
                                       value in critiqueDictionary['critique_Price_list']]
            
            #find temporary updated critique list    
            tempCritiqueList = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRestaurant)
            
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict']\
                                                               if item not in tempCritiqueList]
            
            #CHECK IF EXHAUSTED, if so clear out all restaurants 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            
            else:
                #Update the critiqued restaurant list
                critiqueDictionary['critique_Res_list'] = tempCritiqueList.copy()

                #Record the list of restaurants critiqued so far based on price 
                critiqueDictionary['critiqued_Res_AccrdPrice_list'] = updateExtendList(critiqueDictionary['critiqued_Res_AccrdPrice_list'], \
                                                           listCritiqueRestaurant)
                
                #Updating the critiqued items to list
                print('Saving critiqued items at index: ', listCritiqueRestaurant[:5], '...')

                ##Update current valid set , filter out critiqued items, sequence must remain the same
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict']\
                                                                   if item not in tempCritiqueList]
            
        #Scenario 6 negatively critique rating & positively critique rating: both goes up 
        #"I don't want ratings this low?" "I don't want restuarants with rating below XXX" "I want restaurants with rating above XXX"
        #I will critique the ratings below the current rating or specific rating 
        if 'rating' in critique_feature:
            currentRating = float(recommendResDict['recBusinessJson']['rating'])
            
            if critique_value.lower() == 'current_rating':
                critique_value = currentRating
        
            #Critiquing the restaurants that has ratings equal and below this restaurant 
            critiqueRes = [rating/10 for rating in range(0,int(float(critique_value)*10)+1,5)]
            critiqueDictionary['ciritiqued_Rating_list'] = updateExtendList(critiqueDictionary['ciritiqued_Rating_list'], \
                                                                           critiqueRes)
            #If critiqued all price
            if critiqueDictionary['ciritiqued_Rating_list'] == [price/10 for price in range(0,51,5)]:
                print('exhausted list, no better restaurants, recommending the finest restaurants')
                additionalText += '| Exhausted list, no better restaurants, recommending restaurants beyond current rating'

                #reset critiqued restaurant list 
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))] 

                #Clear out the restuarants critiquied by rating 
                critiqueDictionary['critiqued_Res_AccrdStar_list'] = []
                critiqueDictionary['ciritiqued_Rating_list'] = [rating/10 for rating in range(0,int(currentRating*10+1),5)]

            print('critiquing restaurants at rating rangeing at:',critiqueDictionary['ciritiqued_Rating_list'])
            #additionalText += '| critiquing restaurants at rating rangeing at:'+str(critiqueDictionary['ciritiqued_Rating_list'])

            #Get the list of restaurants to critique
            listCritiqueRes = [key  for (key, value) in IS_dictionary.items() if \
                               value in critiqueDictionary['ciritiqued_Rating_list']]
            
            #CHECK EXHAUSTED - Check to see if user critique exhausts the recommendation list
            #Take a temporary list as updated critiquing list 
            tempCritiqueList = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRes)
            
            #Take a temporary list as updated recommendation list 
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                               if item not in tempCritiqueList]
            #Exhausted recommendation list
            if exhaustFallBack(tempRecommendList):
                additionalText += '| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            else:
                #Updating the critiqued items to list
                print('Saving critiqued items at index: ', listCritiqueRes[:5], '...')

                #Update the critiqued restaurant list
                critiqueDictionary['critique_Res_list'] = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRes)

                #Record those as well
                critiqueDictionary['critiqued_Res_AccrdStar_list'] = updateExtendList(critiqueDictionary['critiqued_Res_AccrdStar_list'], \
                                                                                      listCritiqueRes)
                #Update current valid set 
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                                   if item not in critiqueDictionary['critique_Res_list']]
        
        
        #Scenario #7 Critique on Distance, goes up or down, only say positive critiques. Want closer, further, or specific value 
        if 'distance' in critique_feature:
            
            #Get current distance
            currentDistance = float(recommendResDict['recBusinessJson']['distance'].strip('km'))
            
            #If want further distance restaurants
            if 'positive' in critique_posNeg and 'further' in critique_value:
                print('Critiquing distance <=', currentDistance)
                #New requirements for restaurants to be critiqued
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value <= currentDistance]

            #Closer distance, critique value not empty 
            else:
                if (critique_value) and not 'closer' in critique_value:
                    currentDistance = float(critique_value)

                print('Critiquing distance >=', currentDistance)

                #New requirements for restaurants to be critiqued
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value >= currentDistance]


            #Update critique restaurant list, only keep the ones that are critiqued under other features
            critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                                      (item in list(set(listCritiquedRestaurant+\
                                                                        critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdStar_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdCuisine_list'])))] 

            #Update list of restaurants critiqued by the restuarnt distance, entire replacement
            critiqueDictionary['critiqued_Res_AccrdDistance_list'] = listCritiquedRestaurant

            #Update initial valid restuarnt list
            critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict']\
                                                               if item not in critiqueDictionary['critique_Res_list']]

            #Handle case where critiqued everything, there's no restuarnt that satisfy all the critiques 
            if(len(critiqueDictionary['current_user_item_predict']) == 0):
                print('Exhausted sytem')

                #new_Distance= input('There\'s no restaurant that matches your preference within this area, please input a new larger distance')
                additionalText += 'No restaurants that matches your preference within this area, currently default back to 2.0km'
                new_Distance = 2.0
                
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value > new_Distance]

                #Restore original critiqued restaurants under distance feature 
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    (item in list(set(listCritiquedRestaurant + \
                                                      critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdStar_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdCuisine_list'])))]   
                #New critiquing restuarants
                critiqueDictionary['critiqued_Res_AccrdDistance_list'] = listCritiquedRestaurant

                #Update initial valid restuarnt list
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                                   if item not in critiqueDictionary['critique_Res_list']]
        
        #Full fall back 
        if(len(critiqueDictionary['current_user_item_predict']) ==0):
            fallBackSetup(critique_Res_list=critiqueDictionary['critique_Res_list'], \
                              critiqued_Res_AccrdName_list=critiqueDictionary['critiqued_Res_AccrdName_list'], \
                              critiqueDistance=2.0)
            additionalText += ' Ran out of recommendations at the end of the process, reset, filtered out seen restaurants'
            
            #If still no results
            if len(critiqueDictionary['current_user_item_predict']) ==0:
                fallBackSetup(critique_Res_list=[], \
                              critiqued_Res_AccrdName_list=[], \
                              critiqueDistance=2.0)
                additionalText += ' Clearing EVERYTHING'

        #Construct newly returning item
        return_Json = constructRestaurantDict(recommendIndex, critiqueDictionary['current_user_item_predict'],\
                                              df,ID_dictionary,additionalText)

        #Update restaurant information for the GET method
        recommendResDict['recBusinessJson'] = return_Json.copy()

        #Return newly computed object 
        return {'You sent': request_json, 'Result': return_Json}, 201
            
api.add_resource(Business, '/business') # Route_1

if __name__ == '__main__':
     app.run(port='5002')

{'business_id': '2970', 'name': 'Khao San Road', 'cuisine': 'Thai', 'price': '$$', 'rating': '4.0', 'distance': '0.8km', 'addText': ''}
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5003/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Feb/2020 22:56:34] "GET /business HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 22:56:34] "GET /favicon.ico HTTP/1.1" 404 -


### Testing

#### Initialization

In [ ]:
#Need to initialize to have only restaurants within 2.0km recommended, restaurants distance above 2.0 will be critiqued
critique_distance = 2.0

#list of restaurant don't wanted, initialized as restaurants distance >2.0km
critique_Res_list = [key for (key, value) in ID_dictionary.items() if value > critique_distance] 
critique_Price_list = []
ciritiqued_Rating_list = []
#list of categories don't wanted - Accumulated 
critique_Cat_list = []

#Used to fall back, when user inputs are conflicted, modified during the critiquing process
critiqued_Res_AccrdName_list= []
critiqued_Res_AccrdCuisine_list = []
critiqued_Res_AccrdPrice_list = []
critiqued_Res_AccrdStar_list = []
critiqued_Res_AccrdDistance_list = [key for (key, value) in ID_dictionary.items() if value > critique_distance] 

#Categories that explicitly wanted, ONLY 1 FOR NOW
wanted_Category_index = -1
#wanted_Price_list = []

#Recommendation info
recommendIndex = 0  
outPutString = ''

#Initial user item vector for critiquing process 
current_user_item_predict = [item for item in test_user_item_predict if item not in critique_Res_list]

#Construct initial recommendation
recBusinessJson = constructRestaurantDict(recommendIndex, current_user_item_predict,df,ID_dictionary)
print(recBusinessJson)

#Construct constant variables dictionary
variableDictionary = {
    'critique_distance' : critique_distance,
    'wanted_Category_index': wanted_Category_index
    
}

#A dictionary that stores all the lists related with the critiquing process
critiqueDictionary = {
    'critique_Res_list': critique_Res_list,
    'critique_Price_list':critique_Price_list,
    'critique_Cat_list': critique_Cat_list,
    'ciritiqued_Rating_list':ciritiqued_Rating_list,
    'critiqued_Res_AccrdStar_list': critiqued_Res_AccrdStar_list,
    'critiqued_Res_AccrdCuisine_list':critiqued_Res_AccrdCuisine_list,
    'critiqued_Res_AccrdPrice_list':critiqued_Res_AccrdPrice_list,
    'test_user_item_predict': test_user_item_predict,
    'current_user_item_predict': current_user_item_predict,
    'critiqued_Res_AccrdName_list': critiqued_Res_AccrdName_list
    }


#Storing the current recommended restaurant
recommendResDict = {
        'recBusinessJson': recBusinessJson
}

In [ ]:
critique_feature = 'price'
critique_posNeg = 'positive'
critique_value = 'more expensive'
additionalText  = ''

#Scenario 4 - Negative critique restuarant price e.g. "I don't want expensive restaurants" OR positive "I want cheaper"
#Scenario 5 - Positively critique restaurant price, "I want fine dining", wanting more expensive restaurant 
if 'price' in critique_feature:

    #Get current price label range in 1 - 4
    currentPriceLabel = IP_dictionary[critiqueDictionary['current_user_item_predict'][recommendIndex]]

    #Get critique_price_list first
    templist = critiqueDictionary.get('critique_Price_list', [])

    #Update critiqued price list
    if 'positive' in critique_posNeg and critique_value == 'more expensive':
        templist.extend([price for price in range(1,currentPriceLabel+1,1)])

    #Negative or positive, cheaper     
    else:     
        templist.extend([price for price in range(currentPriceLabel,5,1)])

    #update critiqued_price_list by puting it back to the dictionary
    critiqueDictionary['critique_Price_list'] = list(set(templist))

    #Check if critiqued all price range, if exhausted, fall back 
    if len(critiqueDictionary['critique_Price_list']) == 4:
        print('You have exhaused the price range option, showing the most', critique_value, 'option')
        #Send feedback to front end
        additionalText = 'You have exhaused the price range option, showing the most'+ critique_value + 'option'

        #Clear the previous critiqued restuarants based on price out of critiqued restuarant list 
        #POTENTIAL ISSUE HERE, MAY BE ERASING SOME RESTAURANTS CRITIQUED IN OTHER STEPS ... NEED TO RECHECK
        critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                            item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list'] + \
                                             critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                             critiqueDictionary['critiqued_Res_AccrdStar_list'] + \
                                             critiqueDictionary['critiqued_Res_AccrdDistance_list']))]

        #Clear out the restuarants critiquied by price 
        critiqueDictionary['critiqued_Res_AccrdPrice_list'] = []

        #Restore default critiquing price list 
        if critique_value == 'more expensive':
            critiqueDictionary['critique_Price_list'] = [1,2,3]
        else:
            critiqueDictionary['critique_Price_list'] = [2,3,4]  

    #printing and sending message to front end, the critiquing price range
    print('Critiqing price at range:', ['$'*label for label in critiqueDictionary['critique_Price_list'] ])
    additionalText += 'Critiqing price at range:'+str(['$'*label for label in critiqueDictionary['critique_Price_list']])

    #Find the list of restaurants to critique 
    listCritiqueRestaurant = [key  for (key, value) in IP_dictionary.items() if \
                               value in critiqueDictionary['critique_Price_list']]

    #Record the list of restaurants critiqued so far based on price 
    critiqueDictionary['critiqued_Res_AccrdPrice_list'] = updateExtendList(critiqueDictionary['critiqued_Res_AccrdPrice_list'], \
                                               listCritiqueRestaurant)

    #Update the critiqued restaurant list
    critiqueDictionary['critique_Res_list'] = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRestaurant)

    #Updating the critiqued items to list
    print('Saving critiqued items at index: ', listCritiqueRestaurant[:5], '...')

    ##Update current valid set , filter out critiqued items, sequence must remain the same
    critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict']\
                                                       if item not in critiqueDictionary['critique_Res_list']]

    #Check if exhaused list
    critiqueDictionary['current_user_item_predict'], message= checkIfExhaustedList(critiqueDictionary['current_user_item_predict'],\
                                                                           critiqueDictionary['test_user_item_predict'], \
                                                                           critiqueDictionary['critique_Res_list'])
    #Add message
    additionalText += message
    
#Construct newly returning item
return_Json = constructRestaurantDict(recommendIndex, critiqueDictionary['current_user_item_predict'],\
                                      df,ID_dictionary,additionalText)
print(return_Json)

In [ ]:
additionalText 

## Original critiquing logic
need to update updateExtendList function 
need to check when calling exhaust list, if retrieving message 


In [68]:
print('Initial Recommendation')
print('Enter Stop any time to exist loop')
#displayRestaurantInfo(recommendIndex, test_user_item_predict)

while True:
    satisfied = 'None'
    feature = 'None'
    positiveOrNegative = 'None'
    critiqueValue = 'None'
    
    print('Recommending...')
    displayRestaurantInfo(recommendIndex, current_user_item_predict,df,ID_dictionary)
    
    #Current recommendation cuisine type 
    currentCuisineType = df[df['business_num_id'] == current_user_item_predict[recommendIndex]]\
                                                        .categories.unique()[0].split(', ')
    currentPriceLabel = IP_dictionary[current_user_item_predict[recommendIndex]]
    currentRating = IS_dictionary[current_user_item_predict[recommendIndex]]
    currentDistance = ID_dictionary[current_user_item_predict[recommendIndex]]
    
    #First testing cuisine type
    print('\n????????????????')
    
    while satisfied.lower() not in ['yes', 'no', 'stop']: 
        satisfied = input("You Like? ('yes', 'no') ").strip().lower()
    
    if satisfied == 'stop' or satisfied == 'yes':
        print('BYE :)')
        break
        
    #When satisfied is NO, take in feature
    while feature.lower() not in ['name', 'cuisine', 'price', 'distance', 'rating', 'stop']: 
        feature = input("What feature to critique: (name, cuisine, price, rating, distance)")
    
    if feature == 'stop':
        break
    
    #Take in Positive or nagative 
    if feature != 'name':
        while positiveOrNegative.lower() not in ['positive', 'negative', 'stop']:
            positiveOrNegative = input("Positive or negative: ")
    
    if positiveOrNegative == 'stop':
        break
    
    #Only ask for critique value when not critiquing restaurant name, or not negatively critiquing price 
    if feature != 'name' and not(feature == 'price' and positiveOrNegative == 'negative')\
    and not(feature == 'distance' and positiveOrNegative == 'negative'):
    #and not(feature == 'rating' and positiveOrNegative == 'negative')\
     
        #The valid values to be critiuqed that can pass in 
        validCritiqueValueList = []
        #negatively critique current cuisine type, should only enter current cruisine type
        if 'cuisine' in feature and 'negative' in positiveOrNegative:
            validCritiqueValueList = [cat.strip().lower() for cat in currentCuisineType] + ['stop']
            outPutString = '(' + currentCuisineType[0] +')'
        elif 'cuisine' in feature and 'positive' in positiveOrNegative:
            validCritiqueValueList = [cat.strip().lower() for cat in categoryList] 
            categories = list(set(IC_dictionary.keys())) + ['stop']
            outPutString = '(' + categories[0] +categories[1] +categories[2] +'...)'
        #Can only enter cheapter or more expensive
        elif 'price' in feature:
            validCritiqueValueList = ['cheaper', 'more expensive','stop']
            outPutString = '(cheaper, more expensive)'
        elif 'rating' in feature:
            validCritiqueValueList = [str(star/10) for star in range(0,51,1)] + ['stop', 'current_rating']
            outPutString = '(0 ~ 5.0 with 0.1 increment)'
        #positively critique distance
        elif 'distance' in feature:
            validCritiqueDis = [str(i/10) for i in range(0,int((variableDictionary['maxDistance']+0.5)*10),5)][1:] 
            validCritiqueValueList = ['closer', 'further'] + validCritiqueDis + ['stop']
            outPutString = '(closer, further, or distance in range 0.5 ~' + str(variableDictionary['maxDistance']) + 'in every 0.5km)' 
        #Prompt to ask critique value
        while critiqueValue not in validCritiqueValueList:
            critiqueValue = input("Critique value: " + outPutString).strip().lower()
        if critiqueValue == 'stop':
            break
            
    print('????????????????\n')
    
    #If user starts critiquing, the current showend restaurant will be in critiqued list 
    critique_Res_list.append(current_user_item_predict[recommendIndex])
    critique_Res_list = list(set(critique_Res_list))
    #critique_Res_list = updateList(critique_Res_list, current_user_item_predict[recommendIndex])
    
    #Scenario 1 - Critique Restaurant name 
    if 'name' in feature:
        
        print("Saving critiqued item: ", current_user_item_predict[recommendIndex])
        
        #Save critiqued restaurant to list 
        #critique_Res_list.append(current_user_item_predict[recommendIndex])
        #critique_Res_list = list(set(critique_Res_list))
        #critique_Res_list = updateList(critique_Res_list, current_user_item_predict[recommendIndex])
        
        #update critiqued restaurant list according to name
        #critiqued_Res_AccrdName_list.append(current_user_item_predict[recommendIndex])
        critiqued_Res_AccrdName_list = updateList(critiqued_Res_AccrdName_list, current_user_item_predict[recommendIndex])
        
        current_user_item_predict = current_user_item_predict[recommendIndex+1 :]
        #Handling all items critiqued case
        """BUT I'M NOT COUNTING THE FACTOR THAT THE USER HAD LIKED CATEGORIES"""
        current_user_item_predict, message = checkIfExhaustedList(current_user_item_predict, test_user_item_predict, critique_Res_list)
            
    #Scenario 2 - Negatively critique restaurant features 
    if ('cuisine' in feature.lower()) and ('negative' in positiveOrNegative.lower()):
        
        #Find the correct category name the user want to critique in current recommended item categories
        critiqueValue = [cuisine for cuisine in currentCuisineType if critiqueValue.strip().lower() in cuisine.lower()][0]
        
        critiquied_Cat_Index = IC_dictionary[critiqueValue]
        print('Saving negatively critiqued cuisine type:', critiqueValue, ', cuisine index: ', critiquied_Cat_Index)
        critique_Cat_list.append(critiquied_Cat_Index)
        
        #Handling extreme case - critique categories previously requested 
        if critiquied_Cat_Index == wanted_Category_index:
            #Reset wanted category index
            wanted_Category_index = None
            print('You are critiquing a cuisine type you previously requested\n Fall back to initial preference')
            current_user_item_predict = [item for item in test_user_item_predict if item not in critique_Res_list]
        
        
        #Find list of restaurants to filter out 
        critiquedItemsList = I_C_matrix.getcol(critiquied_Cat_Index).nonzero()[0]
        
        critiqued_Res_AccrdCuisine_list.append(critiquedItemsList)
        critiqued_Res_AccrdCuisine_list =list(set(critique_Res_list))
        
        #Updating the critiqued items to list
        print('Saving critiqued items at index: ', critiquedItemsList[:5], '...')
        critique_Res_list.extend(list(critiquedItemsList))
        critique_Res_list = list(set(critique_Res_list))
        
        #Filter out critiqued items, sequence must remain the same 
        #Update current valid set 
        current_user_item_predict = [item for item in current_user_item_predict if item not in critique_Res_list]

        #Handle case where run out of items! - Fall Back!
        current_user_item_predict,message = checkIfExhaustedList(current_user_item_predict, test_user_item_predict, critique_Res_list)
        
        
    
    #Scenario 3 - Positively critique restaurant cuisine type 
    if 'cuisine' in feature and 'positive' in positiveOrNegative:

        #Find the correct category within all the categories list - assuming exact word typed in 
        """Add ERROR HANDLING HERE"""
        positiveCritiquiCategory = [cuisine for cuisine in categoryList if critiqueValue.strip().lower() in cuisine.lower()][0]
        
        #Retrieve cuisine type index 
        positiveCritiquiedIndex = IC_dictionary[positiveCritiquiCategory]
        
        #Get the preferred category index 
        wanted_Category_index = positiveCritiquiedIndex
        
        #Retrieve items matching item category 
        matchedResList = I_C_matrix.getcol(positiveCritiquiedIndex).nonzero()[0]
        print('matching list:', matchedResList[:10])
        
        #Check if user have previously critiqued - remove it
        if positiveCritiquiedIndex in critique_Cat_list:
            critique_Cat_list.remove(positiveCritiquiedIndex)
            
            #Update critiqued restaurant list according to cuisine type 
            critiqued_Res_AccrdCuisine_list.remove(matchedResList)
            
            #UPDATE CRITIQUE RESTAURANT LIST 
            #In the matched restuarnt list and not critiqued under other criteria will be withdrawn from this list
            critique_Res_list = [item for item in critique_Res_list if item not in matchedResList or \
                                item in list(set(critiqued_Res_AccrdName_list+ critiqued_Res_AccrdCuisine_list+\
                                                 critiqued_Res_AccrdStar_list + critiqued_Res_AccrdDistance_list))]
        
        #Update current valid set, make sure not in critiqued restaurant set 
        current_user_item_predict = [item for item in matchedResList if item not in critique_Res_list]
        
    #Scenario 4 - Negative critique restuarant price e.g. "I don't want expensive restaurants" OR positive "I want cheaper"
    #Does not pass in anything ASSUMING ONLY GOING DOWN
    #Scenario 5 - Positively critique restaurant price, "I want fine dining", wanting more expensive restaurant 
    if 'price' in feature:
        
        #Update critiqued price list
        if 'positive' in positiveOrNegative and critiqueValue == 'more expensive':
            critique_Price_list.extend([price for price in range(1,currentPriceLabel+1,1)])
        #Negative or positive, cheaper     
        else:     
            critique_Price_list.extend([price for price in range(currentPriceLabel,5,1)])
            
        #Deduplicate
        critique_Price_list = list(set(critique_Price_list))
        
        #Check if critiqued all price range, if exhausted, fall back 
        if len(critique_Price_list) == 4:
            print('You have exhaused the price range option, showing the most', critiqueValue, 'option')
            #Clear the previous critiqued restuarants based on price out of critiqued restuarant list 
            #POTENTIAL ISSUE HERE, MAY BE ERASING SOME RESTAURANTS CRITIQUED IN OTHER STEPS ... NEED TO RECHECK
            critique_Res_list = [item for item in test_user_item_predict if\
                                item in list(set(critiqued_Res_AccrdName_list+ critiqued_Res_AccrdCuisine_list+\
                                                 critiqued_Res_AccrdStar_list + critiqued_Res_AccrdDistance_list))]
            
            #Clear out the restuarants critiquied by price 
            critiqued_Res_AccrdPrice_list = []
            
            #Restore default critiquing price list 
            if critiqueValue == 'more expensive':
                critique_Price_list = [1,2,3]
            else:
                critique_Price_list = [2,3,4]  
        
        print('Critiqing price at range:', ['$'*label for label in critique_Price_list])
        
        #Find the list of restaurants to critique 
        listCritiqueRestaurant = [key  for (key, value) in IP_dictionary.items() if value in critique_Price_list]
        
        #Record the list of restaurants critiqued so far based on price 
        #critiqued_Res_AccrdPrice_list.extend(listCritiqueRestaurant)
        #critiqued_Res_AccrdPrice_list = list(set(critiqued_Res_AccrdPrice_list))
        
        critiqued_Res_AccrdPrice_list = updateExtendList(critiqued_Res_AccrdPrice_list, listCritiqueRestaurant)
        
        #Update the critiqued restaurant list
        #critique_Res_list.extend(listCritiqueRestaurant)
        #critique_Res_list = list(set(critique_Res_list))
        critique_Res_list = updateExtendList(critique_Res_list, listCritiqueRestaurant)
        
        
        #Updating the critiqued items to list
        print('Saving critiqued items at index: ', listCritiqueRestaurant[:5], '...')
        
        #Filter out critiqued items, sequence must remain the same 
        #Update current valid set 
        current_user_item_predict = [item for item in current_user_item_predict if item not in critique_Res_list]
        
        #Check if exhaused list
        current_user_item_predict, message = checkIfExhaustedList(current_user_item_predict, test_user_item_predict, critique_Res_list)
        
        #Recommended
        print('Re-recommending...')
        
    #Scenario 6 negatively critique rating & positively critique rating: both goes up 
    #"I don't want ratings this low?" "I don't want restuarants with rating below XXX" "I want restaurants with rating above XXX"
    #I will critique the ratings below the current rating or specific rating 
    if 'rating' in feature:
        #Critiquing the restaurants that has ratings equal and below this restaurant 
        #if 'negative' in positiveOrNegative:
            #ciritiqued_Rating_list.extend([rating/10 for rating in range(0,int(currentRating*10+1),5)])
        #else:
        
        if critique_value.lower() == 'current_rating':
                critique_value = currentRating

        ciritiqued_Rating_list.extend([rating/10 for rating in range(0,int(float(critiqueValue)*10)+1,5)])   
        ciritiqued_Rating_list = list(set(ciritiqued_Rating_list))
            
        #If critiqued all price
        if ciritiqued_Rating_list == [price/10 for price in range(0,int(5.0 *10 +1),5)]:
            print('exhausted list, no better restaurants, recommending he finest restaurants')
            #reset critiqued restaurant list 
            critique_Res_list = [item for item in test_user_item_predict if\
                                item in list(set(critiqued_Res_AccrdName_list+critiqued_Res_AccrdCuisine_list+\
                                                critiqued_Res_AccrdPrice_list+critiqued_Res_AccrdDistance_list))] 

            #Clear out the restuarants critiquied by rating 
            critiqued_Res_AccrdStar_list = []

            ciritiqued_Rating_list = [rating/10 for rating in range(0,int(currentRating*10+1),5)]

        print('critiquing restaurants at rating rangeing at:',ciritiqued_Rating_list)

        #Get the list of restaurants to critique
        listCritiqueRes = [key  for (key, value) in IS_dictionary.items() if value in ciritiqued_Rating_list]

        #Updating the critiqued items to list
        print('Saving critiqued items at index: ', listCritiqueRes[:5], '...')

        #Update the critiqued restaurant list
        #critique_Res_list.extend(listCritiqueRes)
        #critique_Res_list = list(set(critique_Res_list))
        critique_Res_list = updateExtendList(critique_Res_list, listCritiqueRes)
        
        #Record those as well
        #critiqued_Res_AccrdStar_list.extend(listCritiqueRes)
        #critiqued_Res_AccrdStar_list = list(set(critiqued_Res_AccrdStar_list))
        critiqued_Res_AccrdStar_list = updateExtendList(critiqued_Res_AccrdStar_list, listCritiqueRes)

        #Update current valid set 
        current_user_item_predict = [item for item in current_user_item_predict if item not in critique_Res_list]
        
        
    #Scenario #7 Critique on Distance, goes up or down, only say positive critiques
    #Want closer, further, or specific value 
    if 'distance' in feature:
        #If want further distance restaurants
        if 'positive' in positiveOrNegative and 'further' in critiqueValue:
            print('Critiquing distance <=', currentDistance)

            #New requirements for restaurants to be critiqued
            listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value <= currentDistance]
        
        #Closer distance
        else:
            if (critiqueValue != 'None') and not 'closer' in critiqueValue:
                currentDistance = int(critiqueValue)
                
            print('Critiquing distance >=', currentDistance)
            
            #New requirements for restaurants to be critiqued
            listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value >= currentDistance]
        

        #Update critique restaurant list, only keep the ones that are critiqued under other features
        critique_Res_list = [item for item in test_user_item_predict if\
                            (item in list(set(listCritiquedRestaurant+critiqued_Res_AccrdPrice_list+\
                                              critiqued_Res_AccrdStar_list+critiqued_Res_AccrdName_list+\
                                                critiqued_Res_AccrdCuisine_list)))] 
        
        #Update list of restaurants critiqued by the restuarnt distance, entire replacement
        critiqued_Res_AccrdDistance_list= listCritiquedRestaurant
        
        #Update initial valid restuarnt list
        current_user_item_predict = [item for item in  current_user_item_predict if item not in critique_Res_list]
        
        #Handle case where critiqued everything, there's no restuarnt that satisfy all the critiques 
        if(len(current_user_item_predict) == 0):
            print('Exhausted sytem')
            
            new_Distance= input('There\'s no restaurant that matches your preference within this area, please input a new larger distance')

            listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value > new_Distance]
            
            #Restore original critiqued restaurants under distance feature 
            critique_Res_list = [item for item in test_user_item_predict if\
                                (item in list(set(listCritiquedRestaurant+critiqued_Res_AccrdPrice_list\
                                                  +critiqued_Res_AccrdStar_list+critiqued_Res_AccrdName_list+\
                                                    critiqued_Res_AccrdCuisine_list)))] 
            
            #New critiquing restuarants
            critiqued_Res_AccrdDistance_list= listCritiquedRestaurant
        
            #Update initial valid restuarnt list
            current_user_item_predict = [item for item in  current_user_item_predict if item not in critique_Res_list]
        
    

Initial Recommendation
Enter Stop any time to exist loop
Recommending...


NameError: name 'recommendIndex' is not defined

In [ ]:
current_user_item_predict

In [ ]:
merge = listCritiquedRestaurant+critiqued_Res_AccrdPrice_list+ critiqued_Res_AccrdStar_list+critiqued_Res_AccrdName_list + critiqued_Res_AccrdCuisine_list
a = 4139
4139 not in critiqued_Res_AccrdDistance_list or 4139 in list(set(merge))

### Requests  Example

In [ ]:
from flask import Flask
from flask_restful import reqparse, abort, Api, Resource

app = Flask(__name__)
api = Api(app)

TODOS = {
    'todo1': {'task': 'build an API'},
    'todo2': {'task': '?????'},
    'todo3': {'task': 'profit!'},
}


def abort_if_todo_doesnt_exist(todo_id):
    if todo_id not in TODOS:
        abort(404, message="Todo {} doesn't exist".format(todo_id))

parser = reqparse.RequestParser()
parser.add_argument('task')


# Todo
# shows a single todo item and lets you delete a todo item
class Todo(Resource):
    def get(self, todo_id):
        abort_if_todo_doesnt_exist(todo_id)
        return TODOS[todo_id]

    def delete(self, todo_id):
        abort_if_todo_doesnt_exist(todo_id)
        del TODOS[todo_id]
        return '', 204

    def put(self, todo_id):
        args = parser.parse_args()
        task = {'task': args['task']}
        TODOS[todo_id] = task
        return task, 201


# TodoList
# shows a list of all todos, and lets you POST to add new tasks
class TodoList(Resource):
    def get(self):
        return TODOS

    def post(self):
        args = parser.parse_args()
        todo_id = int(max(TODOS.keys()).lstrip('todo')) + 1
        todo_id = 'todo%i' % todo_id
        TODOS[todo_id] = {'task': args['task']}
        return TODOS[todo_id], 201


## Actually setup the Api resource routing here

api.add_resource(TodoList, '/todos')
api.add_resource(Todo, '/todos/<todo_id>')


if __name__ == '__main__':
    app.run(debug=False)